In [ ]:
# ===== Cell 0: 環境設定（全セル共通で利用）=====

from __future__ import annotations

from typing import Any, Callable, Dict, Optional

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

# ------------------------
# 実験スイッチ（Notebook全体で共有）
# ------------------------
FMS_THRESHOLD: int = 1            # FMS >= 1 を陽性ラベルとみなす
EPOCH_LEN: int = 30               # 30 / 60 / 120 のいずれか
MODEL_BACKEND: str = "xgb"        # "xgb" / "rf" / "svm"
USE_AP_FOR_K: bool = False         # APベースの best_k で上書きするか      # 表示用
SEED_BASE: int = 20251101

if EPOCH_LEN not in (30, 60, 120):
    raise ValueError("EPOCH_LEN は 30/60/120 から選択してください。")

# ------------------------
# ファイル入出力ルート
# ------------------------
BASE_INPUT_DIR = r"C:\Users\taiki\OneDrive - Science Tokyo\デスクトップ\研究\本実験結果"
BASE_ANALYSIS_DIR = os.path.join(BASE_INPUT_DIR, "ANALYSIS")
OUT_DIR = os.path.join(BASE_ANALYSIS_DIR, "機械学習(MSSQ込み)", f"閾値FMS{FMS_THRESHOLD}_区間{EPOCH_LEN}")
os.makedirs(OUT_DIR, exist_ok=True)

def outpath(filename: str) -> str:
    return os.path.join(OUT_DIR, filename)

print(f"[OUT_DIR] {OUT_DIR}  |  EPOCH_LEN={EPOCH_LEN}s")

# ------------------------
# 対象被験者・時間窓
# ------------------------
SUBJECT_IDS = [
    "10061","10063","10064",
    "10071","10072","10073","10074",
    "10081","10082","10083",
    "10091","10092","10093","10094",
    "10101","10102","10103",
]

BASELINE_EPOCH = 1770               # ベースライン行（必須）
ML_START, ML_END = 1800, 2400       # 学習に使う epoch_start 範囲 [start, end)

# ------------------------
# 描画スタイル
# ------------------------
plt.rcParams.update({
    "figure.dpi": 120, "savefig.dpi": 300,
    "font.size": 20, "axes.titlesize": 26, "axes.labelsize": 22,
    "xtick.labelsize": 20, "ytick.labelsize": 20, "legend.fontsize": 20,
})

# ------------------------
# FMS二値化ヘルパ
# ------------------------
def binarize_fms(series: pd.Series, threshold: Optional[int] = None) -> pd.Series:
    th = FMS_THRESHOLD if threshold is None else int(threshold)
    return (series >= th).astype(int)

# ------------------------
# モデルレジストリ
# ------------------------
ModelBuilder = Callable[..., Any]
MODEL_REGISTRY: Dict[str, Dict[str, Any]] = {}

def register_backend(name: str, params: Dict[str, Any], builder: ModelBuilder) -> None:
    MODEL_REGISTRY[name] = {"params": params, "builder": builder}

def _build_xgb(params: Dict[str, Any], *, scale_pos_weight: Optional[float] = None):
    cfg = params.copy()
    if scale_pos_weight is not None:
        cfg["scale_pos_weight"] = float(scale_pos_weight)
    return xgb.XGBClassifier(**cfg)

def _build_rf(params: Dict[str, Any], **_):
    return RandomForestClassifier(**params)

def _build_svm(params: Dict[str, Any], **_):
    return SVC(**params)

XGB_PARAMS: Dict[str, Any] = dict(
    n_estimators=100,
    eval_metric="logloss",
    subsample=1.0,
    colsample_bytree=1.0,
    n_jobs=1,
    tree_method="hist",
    device="cpu",
    seed=0,
    random_state=0,
)

RF_PARAMS: Dict[str, Any] = dict(
    n_estimators=439,
    max_depth=14,
    min_samples_split=10,
    min_samples_leaf=4,
    max_features="sqrt",
    bootstrap=False,
    class_weight="balanced",
    random_state=SEED_BASE,
    n_jobs=1,
)

SVM_PARAMS: Dict[str, Any] = dict(
    C=1.0,
    kernel="rbf",
    gamma="scale",
    probability=True,
    class_weight="balanced",
    random_state=SEED_BASE,
)

register_backend("xgb", XGB_PARAMS, _build_xgb)
register_backend("rf",  RF_PARAMS,  _build_rf)
register_backend("svm", SVM_PARAMS, _build_svm)

def set_model_backend(name: str) -> None:
    name = name.lower()
    if name not in MODEL_REGISTRY:
        raise KeyError(f"[ERROR] backend '{name}' は未登録: {list(MODEL_REGISTRY.keys())}")
    global MODEL_BACKEND
    MODEL_BACKEND = name

def build_estimator(
    backend: Optional[str] = None,
    *,
    scale_pos_weight: Optional[float] = None,
    overrides: Optional[Dict[str, Any]] = None,
):
    name = (backend or MODEL_BACKEND).lower()
    if name not in MODEL_REGISTRY:
        raise KeyError(f"[ERROR] backend '{name}' は未登録。")
    base = MODEL_REGISTRY[name]["params"].copy()
    if overrides:
        base.update(overrides)
    builder = MODEL_REGISTRY[name]["builder"]
    return builder(base, scale_pos_weight=scale_pos_weight)

def fit_estimator(
    X_train: pd.DataFrame,
    y_train: pd.Series,
    *,
    backend: Optional[str] = None,
    scale_pos_weight: Optional[float] = None,
    overrides: Optional[Dict[str, Any]] = None,
):
    X_train = X_train.astype(np.float32, copy=False)
    y_train = y_train.astype(np.int32, copy=False)
    model = build_estimator(
        backend=backend, scale_pos_weight=scale_pos_weight, overrides=overrides
    )
    model.fit(X_train, y_train)
    return model

def predict_positive_score(model, X: pd.DataFrame) -> np.ndarray:
    X = X.astype(np.float32, copy=False)
    if hasattr(model, "predict_proba"):
        return model.predict_proba(X)[:, 1]
    if hasattr(model, "decision_function"):
        return np.asarray(model.decision_function(X), dtype=float)
    return model.predict(X).astype(float)

MODEL_ID = MODEL_BACKEND.upper()
print(f"[INFO] MODEL_BACKEND={MODEL_ID} / SEED={SEED_BASE} / backends={list(MODEL_REGISTRY.keys())}")


In [ ]:
# ===== Cell 1: データ準備（CSV読込 → EPOCH合成 → SUBJECT_META → 行列出力）=====

import pandas as pd
import numpy as np

# --------------------------------------------
# ① 30秒EPOCH CSVの読み込み・検証
# --------------------------------------------
def subject_csv_path(sid: str) -> str:
    path = os.path.join(BASE_INPUT_DIR, sid, "EPOCH", f"{sid}_epoch.csv")
    if not os.path.exists(path):
        raise FileNotFoundError(f"[Cell1] CSV missing for subject {sid}: {path}")
    return path

dfs = []
for sid in SUBJECT_IDS:
    df = pd.read_csv(subject_csv_path(sid))
    if df.shape[1] < 4:
        raise ValueError(f"[Cell1] {sid}: 列数が不足（>=4 必須）")
    df = df.copy()
    df.columns = list(df.columns[:3]) + [str(c) for c in df.columns[3:]]
    c1, c2, c3 = df.columns[:3]
    df = df.rename(columns={c1: "epoch_start", c2: "epoch_end", c3: "FMS"})
    df["epoch_start"] = pd.to_numeric(df["epoch_start"], errors="coerce").astype("Int64")
    df["epoch_end"]   = pd.to_numeric(df["epoch_end"],   errors="coerce").astype("Int64")
    df["FMS"]         = pd.to_numeric(df["FMS"],         errors="coerce").astype("Int64")
    if df[["epoch_start","epoch_end","FMS"]].isna().any().any():
        raise ValueError(f"[Cell1] {sid}: epoch_start/epoch_end/FMS に NaN")
    df.insert(0, "subject_id", sid)
    dfs.append(df)

combined_raw = pd.concat(dfs, ignore_index=True)
exclude_feats = {"HF_power", "LF_power", "LF_HF_ratio"}
feature_cols_all = [
    c for c in combined_raw.columns
    if c not in {"subject_id","epoch_start","epoch_end","FMS"} and c not in exclude_feats
]
if not feature_cols_all:
    raise RuntimeError("[Cell1] 特徴量列が0です。列名や除外設定を確認してください。")

print(f"[Cell1] Loaded subjects={len(SUBJECT_IDS)}, rows={len(combined_raw)}, features(after drop)={len(feature_cols_all)}")

# --------------------------------------------
# ② EPOCH_LEN 秒への合成 + baseline差分 + ラベル生成
# --------------------------------------------
if (ML_END - ML_START) % EPOCH_LEN != 0:
    raise ValueError(f"[Cell1] ML window {ML_END-ML_START} が EPOCH_LEN={EPOCH_LEN} で割り切れません。")

rows_per_bin = EPOCH_LEN // 30
df_out_list = []

for sid, sdf in combined_raw.groupby("subject_id", sort=False):
    base_row = sdf.loc[sdf["epoch_start"] == BASELINE_EPOCH]
    if len(base_row) != 1:
        raise ValueError(f"[Cell1] {sid}: baseline epoch_start=={BASELINE_EPOCH} が見つからない")
    base_vals = base_row[feature_cols_all].astype(float).iloc[0]
    if base_vals.isna().any():
        raise ValueError(f"[Cell1] {sid}: baselineにNaN -> {base_vals.index[base_vals.isna()].tolist()}")

    sdf_ml = sdf[(sdf["epoch_start"] >= ML_START) & (sdf["epoch_start"] < ML_END)].copy()
    if sdf_ml.empty:
        raise ValueError(f"[Cell1] {sid}: ML window [{ML_START},{ML_END}) が空です。")

    sdf_ml["bin_start"] = ML_START + ((sdf_ml["epoch_start"] - ML_START) // EPOCH_LEN) * EPOCH_LEN
    sdf_ml["bin_end"]   = sdf_ml["bin_start"] + EPOCH_LEN

    bin_counts = sdf_ml.groupby(["bin_start","bin_end"]).size()
    complete_bins = bin_counts[bin_counts == rows_per_bin].index
    sdf_ml = sdf_ml.set_index(["bin_start","bin_end"]).loc[complete_bins].reset_index()
    if sdf_ml.empty:
        raise ValueError(f"[Cell1] {sid}: EPOCH_LEN={EPOCH_LEN} で完全なbinが無い")

    agg_dict = {c: "mean" for c in feature_cols_all}
    agg_dict["FMS"] = "mean"
    g = sdf_ml.groupby(["subject_id","bin_start","bin_end"], as_index=False).agg(agg_dict)

    g_features = g[feature_cols_all].astype(float) - base_vals.values
    if g_features.isna().any().any():
        bad = g_features.columns[g_features.isna().any()].tolist()
        raise ValueError(f"[Cell1] {sid}: baseline差分後にNaN -> {bad}")

    g_out = pd.concat([g[["subject_id","bin_start","bin_end","FMS"]], g_features], axis=1)
    g_out = g_out.rename(columns={"bin_start":"epoch_start","bin_end":"epoch_end"})
    g_out["label"] = binarize_fms(g_out["FMS"])
    g_out = g_out[["subject_id","epoch_start","epoch_end","FMS","label"] + feature_cols_all]
    df_out_list.append(g_out)

df_ml_epoch = pd.concat(df_out_list, ignore_index=True)

# --------------------------------------------
# ③ SUBJECT_META & MSSQ group
# --------------------------------------------
CANDIDATE_SCORE_PATHS = [
    "/mnt/data/summary_scores.xlsx",
    os.path.join(BASE_ANALYSIS_DIR, "summary_scores.xlsx"),
    os.path.join(BASE_ANALYSIS_DIR, "機械学習", "summary_scores.xlsx"),
    os.path.join(BASE_INPUT_DIR, "summary_scores.xlsx"),
]
score_path = next((p for p in CANDIDATE_SCORE_PATHS if os.path.exists(p)), None)
if score_path is None:
    raise FileNotFoundError("[Cell1] summary_scores.xlsx が見つかりません。")
meta_raw = pd.read_excel(score_path, sheet_name="Summary")

required = ["ID", "MSSQ", "VIMSSQ"]
missing = [c for c in required if c not in meta_raw.columns]
if missing:
    raise ValueError(f"[Cell1] summary_scores.xlsx に必須列がありません -> {missing}")

meta = meta_raw[required].copy()
meta["ID"] = (
    meta["ID"].astype(str).str.strip().str.replace(r"\.0$", "", regex=True)
)
for c in ["MSSQ", "VIMSSQ"]:
    meta[c] = pd.to_numeric(meta[c], errors="raise")

sid_set = set(map(str, SUBJECT_IDS))
meta = meta[meta["ID"].isin(sid_set)].copy()
if meta["ID"].duplicated().any():
    raise ValueError(f"[Cell1] ID 重複 -> {meta.loc[meta['ID'].duplicated(), 'ID'].tolist()}")

MSSQ_THRESHOLD_FIXED = 12.0
meta["MSSQ_group"] = np.where(meta["MSSQ"] >= MSSQ_THRESHOLD_FIXED, "High", "Low")
SUBJECT_META = (
    meta.rename(columns={"ID": "subject_id"})
        .set_index("subject_id")[["MSSQ", "VIMSSQ", "MSSQ_group"]]
        .copy()
)
SUBJECT_META.to_csv(outpath("subject_meta.csv"), encoding="utf-8-sig")
print(f"[Cell1] SUBJECT_META saved -> {outpath('subject_meta.csv')} (source='{score_path}')")

# --------------------------------------------
# ④ 学習行列＆行列保存
# --------------------------------------------
fname_raw = f"ML_DATA_DELTA_{EPOCH_LEN}S_RAW.CSV"
df_ml_epoch.to_csv(outpath(fname_raw), index=False, encoding="utf-8-sig")

X_all = df_ml_epoch[feature_cols_all].copy().astype(float)
y_all = df_ml_epoch["label"].copy().astype(int)
groups = df_ml_epoch["subject_id"].copy()

X_all.to_csv(outpath(f"X_RAW_ALL_{EPOCH_LEN}S.CSV"), index=False, encoding="utf-8-sig")
X_all.to_csv(outpath(f"X_SCALED_ALL_{EPOCH_LEN}S.CSV"), index=False, encoding="utf-8-sig")  # 木系でスケーリング不要
pd.DataFrame({"subject_id": groups, "label": y_all, "FMS_mean": df_ml_epoch["FMS"]}).to_csv(
    outpath(f"Y_AND_GROUPS_{EPOCH_LEN}S.CSV"), index=False, encoding="utf-8-sig"
)

print(f"[Cell1] Saved -> {outpath(fname_raw)} / X_RAW_ALL / X_SCALED_ALL / Y_AND_GROUPS")
print(f"[Cell1] Matrices ready: X_all={X_all.shape}, y_all={y_all.shape}, SUBJECT_META={SUBJECT_META.shape}")


In [ ]:
# ===== Cell 2: モデリング共通ヘルパ（fit / SHAP / 評価）=====

from typing import Dict, Optional, Tuple

import numpy as np
import pandas as pd
import shap
from sklearn.metrics import roc_auc_score, accuracy_score


# --------------------------------------------
# 学習ラッパー（Cell0のレジストリAPIを利用）
# --------------------------------------------
def fit_classifier(
    X_train: pd.DataFrame,
    y_train: pd.Series,
    *,
    backend: Optional[str] = None,
    scale_pos_weight: Optional[float] = None,
    overrides: Optional[Dict[str, Any]] = None,
):
    """
    Cell0 の fit_estimator を直接包む薄いラッパ。
    - SHAP/評価セルから backend を差し替えたい場合のみ backend / overrides を指定する。
    """
    if "fit_estimator" not in globals():
        raise RuntimeError("[Cell2] fit_estimator が未定義です。Cell0 を先に実行してください。")
    X_train = X_train.astype(np.float32, copy=False)
    y_train = y_train.astype(np.int32, copy=False)
    return fit_estimator(
        X_train,
        y_train,
        backend=backend,
        scale_pos_weight=scale_pos_weight,
        overrides=overrides,
    )


# --------------------------------------------
# TreeSHAP ベースの特徴重要度算出
# --------------------------------------------
def compute_train_shap_abs_mean(model, X_ref: pd.DataFrame) -> pd.Series:
    """
    学習データ X_ref 上での平均絶対SHAP値（降順）。
    - XGB/RF 等の木モデルを想定（TreeSHAP）。
    - SVM など非対応モデルでは ValueError を送出する。
    """
    X_ref = X_ref.astype(np.float32, copy=False)

    # 背景データ（最大128行）
    bg_n = min(128, len(X_ref))
    X_bg = X_ref.sample(n=bg_n, random_state=SEED_BASE) if bg_n >= 2 else X_ref

    try:
        explainer = shap.TreeExplainer(
            model,
            data=X_bg,
            model_output="probability",
            feature_perturbation="interventional",
        )
        sv_any = explainer.shap_values(X_ref)
    except Exception:
        # probability指定が非対応な場合に raw へフォールバック
        explainer = shap.TreeExplainer(
            model,
            model_output="raw",
            feature_perturbation="tree_path_dependent",
        )
        sv_any = explainer.shap_values(X_ref)

    # shap_values の戻り値形状を統一（2D: n_samples × n_features）
    classes = getattr(model, "classes_", None)
    pos_idx = int(np.where(classes == 1)[0][0]) if classes is not None and 1 in list(classes) else -1

    if isinstance(sv_any, list):
        sv = sv_any[pos_idx]
    else:
        sv = getattr(sv_any, "values", sv_any)
        sv = np.asarray(sv)
        if sv.ndim == 3:
            sv = sv[..., pos_idx]
        elif sv.ndim == 1:
            sv = sv.reshape(-1, 1)

    if sv.shape[1] != X_ref.shape[1]:
        raise RuntimeError(
            f"[Cell2] SHAP shape mismatch: sv.shape={sv.shape}, X_ref.shape={X_ref.shape}"
        )

    abs_mean = np.mean(np.abs(sv), axis=0)
    return pd.Series(abs_mean, index=X_ref.columns, name="mean_abs").sort_values(ascending=False)


# --------------------------------------------
# 評価ユーティリティ
# --------------------------------------------
def _is_probability_like(scores: np.ndarray) -> bool:
    return np.isfinite(scores).all() and 0.0 <= scores.min() and scores.max() <= 1.0


def evaluate_fold(model, X_test: pd.DataFrame, y_test: pd.Series) -> Dict[str, float]:
    """
    - ROC AUC: 2クラス時のみ。
    - Accuracy: 確率なら 0.5、スコアなら 0.0 を閾値とする（詳細な最適化は別セル）。
    """
    X_test = X_test.astype(np.float32, copy=False)
    scores = predict_positive_score(model, X_test)

    if len(np.unique(y_test)) == 2:
        roc_auc = roc_auc_score(y_test, scores)
    else:
        roc_auc = float("nan")

    thr = 0.5 if _is_probability_like(scores) else 0.0
    pred = (scores >= thr).astype(int)
    acc = accuracy_score(y_test.astype(int), pred)

    return {"roc_auc": float(roc_auc), "accuracy": float(acc)}


print("[Cell2] Modeling helpers ready (fit_classifier / compute_train_shap_abs_mean / evaluate_fold)")


# ===== Section: 特徴重要度と best_k 探索 =====

| Cell | 目的 | 主な出力 (OUT_DIR 配下) |
| ---- | ---- | ----------------------- |
| 3A | LOSO学習でSHAP重要度を算出しランキング化 | `SHAP_FEATURE_RANKING.CSV`, `SHAP_FEATURE_RANKING_LABELED.CSV`, `SHAP_RANKING_ALL.PNG`, `SHAP_TOP8_RANKING.PNG`, `LOSO_METRICS.CSV` |
| 3B | SHAP順の特徴を使って k 本ごとの pooled ROC-AUC を計測し best_k を決定 | `AUC_PER_K.CSV`, `AUC_VS_NUM_FEATURES.PNG`, `best_k` (グローバル変数) |
| 3C | 同様に AUPRC/AP で k を走査し、必要に応じ best_k を APベースで上書きし PR 曲線を出力 | `AUPRC_PER_K.CSV`, `AP_VS_NUM_FEATURES.PNG`, `PR_CURVE_AT_BEST_K.CSV`, `PR_CURVE_AT_BEST_K.PNG` |
| 3D | MSSQ High/Low 各群で in-group LOSO を行い、群別の AUC vs k と best_k を算出 | `AUC_VS_K_BY_GROUP.PNG`, `BEST_K_BY_GROUP.JSON` |


# ===== Cell 3A: SHAPランキング（LOSO学習側のみ）=====

from sklearn.model_selection import LeaveOneGroupOut

logo = LeaveOneGroupOut()
shap_frames = []
metrics_rows = []

for fold_id, (tr_idx, te_idx) in enumerate(logo.split(X_all, y_all, groups), start=1):
    X_tr, X_te = X_all.iloc[tr_idx], X_all.iloc[te_idx]
    y_tr, y_te = y_all.iloc[tr_idx], y_all.iloc[te_idx]
    if len(np.unique(y_tr)) < 2:
        raise RuntimeError(f"[Cell3A] fold{fold_id}: 学習側が単一クラス")

    model = fit_classifier(X_tr, y_tr)
    abs_mean = compute_train_shap_abs_mean(model, X_tr).rename(f"fold{fold_id}")
    shap_frames.append(abs_mean)

    m = evaluate_fold(model, X_te, y_te)
    metrics_rows.append({
        "fold_id": fold_id,
        "test_subject": groups.iloc[te_idx].iloc[0],
        "roc_auc": m["roc_auc"],
        "accuracy": m["accuracy"],
    })

shap_rank = pd.concat(shap_frames, axis=1)
shap_rank["mean_abs"] = shap_rank.mean(axis=1)
shap_rank = shap_rank.sort_values("mean_abs", ascending=False)

shap_rank.to_csv(outpath("SHAP_FEATURE_RANKING.CSV"), encoding="utf-8-sig")
shap_rank.to_csv(outpath("SHAP_FEATURE_RANKING_LABELED.CSV"), encoding="utf-8-sig")
pd.DataFrame(metrics_rows).to_csv(outpath("LOSO_METRICS.CSV"), index=False, encoding="utf-8-sig")

print("[Cell3A] Saved SHAP ranking & LOSO metrics")

plt.figure(figsize=(10, max(5, len(shap_rank)//3)))
plt.barh(shap_rank.index[::-1], shap_rank["mean_abs"][::-1])
plt.xlabel("Mean |SHAP|"); plt.ylabel("Feature"); plt.title("SHAP Ranking (All)")
plt.tight_layout(); plt.savefig(outpath("SHAP_RANKING_ALL.PNG"), dpi=300); plt.close()

topk = shap_rank.head(8).iloc[::-1]
plt.figure(figsize=(12, 7))
ax = plt.gca()
ax.barh(topk.index, topk["mean_abs"])
mx = float(topk["mean_abs"].max()) if len(topk) else 1.0
ax.set_xlim(0, mx * 1.08)
ax.set_xlabel("Mean |SHAP value|", fontsize=26)
ax.set_ylabel("Feature", fontsize=26)
ax.tick_params(axis="both", labelsize=22)
ax.set_title("Top-8 SHAP Feature Ranking", fontsize=34, pad=10)
plt.tight_layout()
plt.savefig(outpath("SHAP_TOP8_RANKING.PNG"), dpi=300)
plt.close()


In [ ]:
# ===== Cell 3A (Legacy): SHAPランキング（XGB pred_contribs版） =====
# - XGBoost専用（pred_contribs=True）でSHAP値を算出
# - 生成物は現行セルと同名ファイルに上書き保存

from sklearn.model_selection import LeaveOneGroupOut
from xgboost import XGBClassifier
import xgboost as xgb

logo = LeaveOneGroupOut()
shap_frames = []
metrics_rows = []

for fold_id, (tr_idx, te_idx) in enumerate(logo.split(X_all, y_all, groups), start=1):
    X_tr, X_te = X_all.iloc[tr_idx], X_all.iloc[te_idx]
    y_tr, y_te = y_all.iloc[tr_idx], y_all.iloc[te_idx]
    if len(np.unique(y_tr)) < 2:
        raise RuntimeError(f"[Cell3A-legacy] fold{fold_id}: 学習側が単一クラスです。")

    model = XGBClassifier(
        n_estimators=100,
        eval_metric="logloss",
        subsample=1.0,
        colsample_bytree=1.0,
        n_jobs=1,
        tree_method="hist",
        device="cpu",
        seed=0,
        random_state=0,
    )
    model.fit(X_tr.astype(np.float32), y_tr.astype(np.int32))

    dm = xgb.DMatrix(X_tr.astype(np.float32), feature_names=list(X_tr.columns))
    contribs = model.get_booster().predict(dm, pred_contribs=True)  # (n_samples, n_features+1)
    shap_vals = contribs[:, :-1]                                   # 最後の列はバイアス項
    abs_mean = np.abs(shap_vals).mean(axis=0)
    shap_frames.append(pd.Series(abs_mean, index=X_tr.columns, name=f"fold{fold_id}"))

    m = evaluate_fold(model, X_te, y_te)
    metrics_rows.append({
        "fold_id": fold_id,
        "test_subject": groups.iloc[te_idx].iloc[0],
        "roc_auc": m["roc_auc"],
        "accuracy": m["accuracy"],
    })

shap_rank = pd.concat(shap_frames, axis=1)
shap_rank["mean_abs"] = shap_rank.mean(axis=1)
shap_rank = shap_rank.sort_values("mean_abs", ascending=False)

# ★ 既存ファイル名へ上書き保存（現行版と同じパス）
shap_rank.to_csv(outpath("SHAP_FEATURE_RANKING.CSV"), encoding="utf-8-sig")
shap_rank.to_csv(outpath("SHAP_FEATURE_RANKING_LABELED.CSV"), encoding="utf-8-sig")
pd.DataFrame(metrics_rows).to_csv(outpath("LOSO_METRICS.CSV"), index=False, encoding="utf-8-sig")

print("[Cell3A-legacy] SHAP_FEATURE_RANKING*.CSV を旧ロジックで上書きしました。")


In [ ]:
# ===== CELLA2: SHAP全体ランクへの相関管理（|r|>0.90, Pearson）＋バックアップ＆上書き =====
# 前提:
#  - outpath(name: str) -> str が定義済みであること
#  - X_all: pd.DataFrame がメモリ上に存在すること（全特徴の数値列を含む）
#  - 既に Cell 3A を実行し、outpath("SHAP_FEATURE_RANKING.CSV") が存在すること
#
# 入出力仕様:
#  - 入力:  SHAP_FEATURE_RANKING.CSV（Cell3A 生成物）
#  - 出力:  同名 CSV を上書き（相関管理後の行のみ残す）
#            ただし上書き前の入力CSVは SHAP_FEATURE_RANKING_BEFORE_CORR.CSV に保存
#          画像も同様に、既存PNGを *_BEFORE_CORR.PNG に退避し、新たに再描画して上書き
#
# バックアップ対象画像（存在するもののみ退避）:
#  - SHAP_RANKING_ALL.PNG      -> SHAP_RANKING_ALL_BEFORE_CORR.PNG
#  - SHAP_TOP8_RANKING.PNG     -> SHAP_TOP8_RANKING_BEFORE_CORR.PNG
#
# 可視化規約（ユーザー指定）:
#  - タイトル 30, 軸ラベル 24, 目盛 20, 凡例 20
#  - 線幅 linewidth=1.5
#  - 画像 dpi=300, tight_layout
#
# 注意:
#  - 本セルは「リーク回避なし」である（X_all 全体で相関を計算）．

import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ---------------- ユーティリティ ----------------
def _assert(cond: bool, msg: str):
    if not cond:
        raise RuntimeError(msg)

def _backup_file(src_path: str, dst_path: str):
    """ファイルが存在すれば dst_path へコピーする（存在しなければ何もしない）"""
    try:
        if os.path.exists(src_path):
            shutil.copy2(src_path, dst_path)
            print(f"[OK] Backup -> {os.path.basename(dst_path)}")
        else:
            print(f"[SKIP] Not found (no backup): {os.path.basename(src_path)}")
    except Exception as e:
        raise RuntimeError(f"[ERROR] Backup failed: {src_path} -> {dst_path}: {e}")

def _corr_prune_global(
    X: pd.DataFrame,
    ranked_features: list[str],
    method: str = "pearson",
    thr: float = 0.90,
    min_var: float = 0.0,
):
    """
    全体ランク ranked_features（降順）に対し，|r|>thr で冗長特徴を間引く（リーク回避なし）．
    戻り値:
        kept_order: 採択順（list[str]）
        dropped_info: list[dict]  # {"feature", "rep", "abs_corr"}
    """
    # 数値列に限定
    num_cols = X.select_dtypes(include=[np.number]).columns.tolist()
    ranked = [f for f in ranked_features if f in num_cols]
    _assert(len(ranked) > 0, "[CELLA2] 数値列に該当する特徴が無い．")

    # 定数/ほぼ定数列の除外（相関NaN回避）
    var = X[ranked].var(axis=0, ddof=1).fillna(0.0)
    ranked = [f for f in ranked if var.get(f, 0.0) > min_var]
    _assert(len(ranked) > 0, "[CELLA2] 有効分散のある特徴が無い．")

    # 相関行列（pairwise complete）
    R = X[ranked].corr(method=method)

    kept, dropped_info = [], []
    for f in ranked:
        # 既採択との最大相関を評価
        if kept:
            abs_corrs = [(k, float(abs(R.loc[f, k]))) for k in kept if pd.notna(R.loc[f, k])]
            if abs_corrs:
                kstar, max_abs = max(abs_corrs, key=lambda t: t[1])
            else:
                kstar, max_abs = None, 0.0
        else:
            kstar, max_abs = None, 0.0

        if max_abs > thr:
            dropped_info.append({"feature": f, "rep": kstar, "abs_corr": max_abs})
        else:
            kept.append(f)

    return kept, dropped_info

# ---------------- 入力の存在確認＆バックアップ ----------------
rank_csv = outpath("SHAP_FEATURE_RANKING.CSV")
_assert(os.path.exists(rank_csv), "[CELLA2] 入力CSVが存在しない: SHAP_FEATURE_RANKING.CSV")

# 既存PNGのパス
png_all = outpath("SHAP_RANKING_ALL.PNG")
png_top = outpath("SHAP_TOP8_RANKING.PNG")

# バックアップ先
rank_csv_bak = outpath("SHAP_FEATURE_RANKING_BEFORE_CORR.CSV")
png_all_bak  = outpath("SHAP_RANKING_ALL_BEFORE_CORR.PNG")
png_top_bak  = outpath("SHAP_TOP8_RANKING_BEFORE_CORR.PNG")

print("[INFO] CorrMgmt(global/no-leak): method=pearson, thr=0.90, min_var=0.0")
_backup_file(rank_csv, rank_csv_bak)
_backup_file(png_all, png_all_bak)
_backup_file(png_top, png_top_bak)

# ---------------- ランキング読み込み ----------------
shap_rank = pd.read_csv(rank_csv, index_col=0, encoding="utf-8-sig")
_assert("mean_abs" in shap_rank.columns, "[CELLA2] shap_rank に 'mean_abs' 列が無い．Cell3A の完了が必要である．")

# 全体ランク（降順）
ranked_features = shap_rank.sort_values("mean_abs", ascending=False).index.tolist()
print(f"[INFO] Ranked features (before corr): n={len(ranked_features)}")

# ---------------- 相関管理の実施（X_all 全体で計算） ----------------
_assert("X_all" in globals(), "[CELLA2] X_all が未定義である．")
kept_order, dropped_info = _corr_prune_global(
    X=X_all,
    ranked_features=ranked_features,
    method="pearson",
    thr=0.90,
    min_var=0.0,
)
print(f"[OK] CorrMgmt: kept={len(kept_order)}, dropped={len(dropped_info)}")

# ---------------- CSVの上書き保存（相関管理後の行のみ） ----------------
# 元の shap_rank から、kept のみを抜粋して 'mean_abs' 降順を維持
shap_after = shap_rank.loc[kept_order].sort_values("mean_abs", ascending=False)
shap_after.to_csv(rank_csv, encoding="utf-8-sig")
print(f"[OK] Overwritten -> {os.path.basename(rank_csv)}")

# 監査用（代表→吸収）のCSVも出しておくと便利である（任意だが有用）
if dropped_info:
    corr_groups_path = outpath("CORR_GROUPS.CSV")
    rows = []
    for d in dropped_info:
        rows.append({"representative": d["rep"], "absorbed_feature": d["feature"], "abs_corr_with_rep": d["abs_corr"]})
    pd.DataFrame(rows).to_csv(corr_groups_path, index=False, encoding="utf-8-sig")
    print(f"[OK] Saved -> {os.path.basename(corr_groups_path)}")

# ---------------- 可視化（上書き出力；規約のフォントサイズを適用） ----------------
def _plot_rank_all(df_sorted: pd.DataFrame, path_png: str, title: str):
    plt.figure(figsize=(10, max(5, len(df_sorted)//3)))
    ax = plt.gca()
    ax.barh(df_sorted.index[::-1], df_sorted["mean_abs"][::-1])
    ax.set_xlabel("Mean |SHAP value|", fontsize=24)
    ax.set_ylabel("Feature", fontsize=24)
    ax.tick_params(axis="both", labelsize=20, width=1.5)
    ax.set_title(title, fontsize=30, pad=10)
    for spine in ax.spines.values():
        spine.set_linewidth(1.5)
    plt.tight_layout()
    plt.savefig(path_png, dpi=300)
    plt.close()

def _plot_rank_topk(df_sorted: pd.DataFrame, path_png: str, k: int = 8):
    topk = df_sorted.head(k).iloc[::-1]
    plt.figure(figsize=(12, 7))
    ax = plt.gca()
    ax.barh(topk.index, topk["mean_abs"])
    mx = float(topk["mean_abs"].max()) if len(topk) else 1.0
    ax.set_xlim(0, mx * 1.08)
    ax.set_xlabel("Mean |SHAP value|", fontsize=24)
    ax.set_ylabel("Feature", fontsize=24)
    ax.tick_params(axis="both", labelsize=20, width=1.5)
    ax.set_title(f"Top-{k} SHAP Feature Ranking (after corr |r|>0.90)", fontsize=30, pad=10)
    for spine in ax.spines.values():
        spine.set_linewidth(1.5)
    plt.tight_layout()
    plt.savefig(path_png, dpi=300)
    plt.close()

# 再描画（上書き）
_plot_rank_all(shap_after.sort_values("mean_abs", ascending=False), png_all, title="SHAP Ranking (All, after corr |r|>0.90)")
_plot_rank_topk(shap_after.sort_values("mean_abs", ascending=False), png_top, k=8)
print(f"[OK] Overwritten -> {os.path.basename(png_all)}, {os.path.basename(png_top)}")

print("[DONE] CELLA2 完了（全体ランクに相関管理を適用し、CSV/PNGをバックアップの上で上書きした）")


In [ ]:
# ===== Cell 3B: 全kで pooled ROC-AUC → best_k 決定 =====

from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import roc_auc_score

rank_df = pd.read_csv(outpath("SHAP_FEATURE_RANKING.CSV"), index_col=0, encoding="utf-8-sig")
feature_order = [f for f in rank_df.index if f in X_all.columns]
if not feature_order:
    raise RuntimeError("[Cell3B] ランキング上位特徴が X_all に存在しません。")

ks = list(range(len(feature_order), 0, -1))
logo = LeaveOneGroupOut()
auc_list = []

for k in ks:
    feats = feature_order[:k]
    X = X_all[feats].astype(np.float32)
    y = y_all.values
    g = groups.values

    y_true_all, proba_all = [], []
    for tr_idx, te_idx in logo.split(X, y, g):
        X_tr, X_te = X.iloc[tr_idx], X.iloc[te_idx]
        y_tr, y_te = y[tr_idx], y[te_idx]
        if len(np.unique(y_tr)) < 2:
            raise RuntimeError("[Cell3B] 学習foldが単一クラス。閾値/期間の見直しが必要です。")
        model = fit_classifier(X_tr, pd.Series(y_tr))
        proba = predict_positive_score(model, X_te)
        y_true_all.append(y_te); proba_all.append(proba)

    y_true_k = np.concatenate(y_true_all)
    proba_k = np.concatenate(proba_all)
    if len(np.unique(y_true_k)) < 2:
        raise RuntimeError("[Cell3B] pooled 真値が単一クラスで AUC 計算不可。")
    auc_list.append(float(roc_auc_score(y_true_k, proba_k)))

pd.DataFrame({"k": ks, "auc_pooled": auc_list}).to_csv(outpath("AUC_PER_K.CSV"), index=False, encoding="utf-8-sig")
auc_array = np.asarray(auc_list, dtype=float)
best_idx = int(np.nanargmax(auc_array))
best_k = ks[best_idx]
best_auc = auc_list[best_idx]
print(f"[Cell3B] Best k (AUC) = {best_k}, AUC={best_auc:.3f}")
globals()["best_k"] = best_k 

plt.figure(figsize=(12, 7))
ax = plt.gca()
ax.plot(ks, auc_list, marker='o', linewidth=1.5)
ax.scatter([best_k], [best_auc], s=180, color="red", zorder=5)
ax.annotate(f"Max AUC = {best_auc:.3f} (k={best_k})", xy=(best_k, best_auc),
            xytext=(best_k, best_auc + 0.02), ha="center", va="bottom",
            fontsize=20, color="red")
ax.invert_xaxis()
ax.set_xlabel("Number of Features (k)", fontsize=26)
ax.set_ylabel("ROC AUC (pooled)", fontsize=26)
ax.tick_params(axis="both", labelsize=22)
ax.set_title("AUC vs Number of Features", fontsize=34, pad=10)
ax.grid(True, alpha=0.4)
plt.tight_layout()
plt.savefig(outpath("AUC_VS_NUM_FEATURES.PNG"), dpi=300)
plt.close()


In [ ]:
# ===== Cell 3C: 全kで pooled AUPRC → best_k (AP) 決定 ＋ PR曲線 =====

from sklearn.metrics import average_precision_score, precision_recall_curve, auc
from sklearn.model_selection import LeaveOneGroupOut

rank_df = pd.read_csv(outpath("SHAP_FEATURE_RANKING.CSV"), index_col=0, encoding="utf-8-sig")
feature_order = [f for f in rank_df.index if f in X_all.columns]
if not feature_order:
    raise RuntimeError("[Cell3C] ランキング上位特徴が X_all に存在しません。")

ks = list(range(len(feature_order), 0, -1))
logo = LeaveOneGroupOut()
ap_list, prauc_list, pi_list = [], [], []

for k in ks:
    feats = feature_order[:k]
    X = X_all[feats].astype(np.float32)
    y = y_all.values
    g = groups.values

    y_true_all, proba_all = [], []
    for tr_idx, te_idx in logo.split(X, y, g):
        X_tr, X_te = X.iloc[tr_idx], X.iloc[te_idx]
        y_tr, y_te = y[tr_idx], y[te_idx]
        if len(np.unique(y_tr)) < 2:
            raise RuntimeError("[Cell3C] 学習foldが単一クラス。")
        model = fit_classifier(X_tr, pd.Series(y_tr))
        proba = predict_positive_score(model, X_te)
        y_true_all.append(y_te); proba_all.append(proba)

    y_true_k = np.concatenate(y_true_all)
    proba_k = np.concatenate(proba_all)
    if len(np.unique(y_true_k)) < 2:
        raise RuntimeError("[Cell3C] pooled 真値が単一クラスで AUPRC 計算不可。")

    ap = float(average_precision_score(y_true_k, proba_k))
    prec, rec, _ = precision_recall_curve(y_true_k, proba_k)
    prauc = float(auc(rec, prec))
    ap_list.append(ap)
    prauc_list.append(prauc)
    pi_list.append(float((y_true_k == 1).mean()))

pd.DataFrame({"k": ks, "ap": ap_list, "prauc": prauc_list, "pi": pi_list}).to_csv(
    outpath("AUPRC_PER_K.CSV"), index=False, encoding="utf-8-sig"
)

ap_array = np.asarray(ap_list, dtype=float)
best_idx_ap = int(np.nanargmax(ap_array))
best_k_ap = ks[best_idx_ap]
print(f"[Cell3C] Best k (AP) = {best_k_ap}, AP={ap_list[best_idx_ap]:.3f}")

plt.figure(figsize=(12, 7))
ax = plt.gca()
ax.plot(ks, ap_list, marker='o', linewidth=1.5, label="AP")
ax.scatter([best_k_ap], [ap_list[best_idx_ap]], s=150, color="red", zorder=5)
ax.invert_xaxis()
ax.set_xlabel("Number of Features (k)", fontsize=26)
ax.set_ylabel("Average Precision", fontsize=26)
ax.tick_params(axis="both", labelsize=22)
ax.set_title("AP vs Number of Features", fontsize=34, pad=10)
ax.grid(True, alpha=0.4)
plt.tight_layout()
plt.savefig(outpath("AP_VS_NUM_FEATURES.PNG"), dpi=300)
plt.close()

if USE_AP_FOR_K:
    globals()["best_k"] = best_k_ap
    print(f"[Cell3C] USE_AP_FOR_K=True → best_k を {best_k_ap} に上書き")
else:
    print("[Cell3C] USE_AP_FOR_K=False → Cell3Bの best_k を維持")

feats_best = feature_order[:globals()["best_k"]]
logo = LeaveOneGroupOut()
y_true_best, proba_best = [], []
for tr_idx, te_idx in logo.split(X_all[feats_best], y_all.values, groups.values):
    X_tr, X_te = X_all[feats_best].iloc[tr_idx], X_all[feats_best].iloc[te_idx]
    y_tr, y_te = y_all.values[tr_idx], y_all.values[te_idx]
    if len(np.unique(y_tr)) < 2:
        raise RuntimeError("[Cell3C] best_k fold が単一クラス。")
    model = fit_classifier(X_tr, pd.Series(y_tr))
    proba = predict_positive_score(model, X_te)
    y_true_best.append(y_te); proba_best.append(proba)

y_true_best = np.concatenate(y_true_best)
proba_best = np.concatenate(proba_best)
prec, rec, thr = precision_recall_curve(y_true_best, proba_best)
ap_best = float(average_precision_score(y_true_best, proba_best))
prauc_best = float(auc(rec, prec))
pi_best = float((y_true_best == 1).mean())

pd.DataFrame({"recall": rec, "precision": prec, "threshold": np.r_[np.nan, thr]}).to_csv(
    outpath("PR_CURVE_AT_BEST_K.CSV"), index=False, encoding="utf-8-sig"
)

plt.figure(figsize=(10, 7))
ax = plt.gca()
ax.step(rec, prec, where="post", linewidth=1.5,
        label=f"PR (AP={ap_best:.3f}, PR-AUC={prauc_best:.3f})")
ax.axhline(pi_best, linestyle="--", linewidth=1.5, label=f"Baseline π={pi_best:.3f}", alpha=0.8)
ax.set_xlabel("Recall", fontsize=24)
ax.set_ylabel("Precision", fontsize=24)
ax.tick_params(axis="both", labelsize=20)
ax.set_title(f"Precision–Recall at best k = {globals()['best_k']}", fontsize=30, pad=10)
ax.set_xlim([0.0, 1.0]); ax.set_ylim([0.0, 1.05])
ax.grid(True, alpha=0.4)
ax.legend(fontsize=20)
plt.tight_layout()
plt.savefig(outpath("PR_CURVE_AT_BEST_K.PNG"), dpi=300)
plt.close()


In [ ]:
# ===== Cell 3D: MSSQ群別 AUC vs k（in-group LOSO）=====

from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import roc_auc_score
import json

req_vars = ["X_all", "y_all", "groups", "SUBJECT_META"]
missing = [v for v in req_vars if v not in globals()]
if missing:
    raise RuntimeError(f"[Cell3D] 必要変数が未定義: {missing}")
if "MSSQ_group" not in SUBJECT_META.columns:
    raise RuntimeError("[Cell3D] SUBJECT_META に MSSQ_group 列が必要です。")

rank_df = pd.read_csv(outpath("SHAP_FEATURE_RANKING.CSV"), index_col=0, encoding="utf-8-sig")
feature_order = [f for f in rank_df.index if f in X_all.columns]
if not feature_order:
    raise RuntimeError("[Cell3D] ランキング上位特徴が存在しません。")

fair_groups = groups.astype(str).map(SUBJECT_META["MSSQ_group"])
if fair_groups.isna().any():
    raise RuntimeError("[Cell3D] SUBJECT_META に存在しない subject_id があります。")

ks = list(range(len(feature_order), 0, -1))
logo = LeaveOneGroupOut()

def _in_group_auc(mask):
    aucs = []
    for k in ks:
        feats = feature_order[:k]
        X = X_all.loc[mask, feats].astype(np.float32)
        y = y_all.loc[mask].values
        g = groups.loc[mask].values
        if len(np.unique(y)) < 2:
            aucs.append(np.nan); continue
        y_true_all, proba_all = [], []
        for tr_idx, te_idx in logo.split(X, y, g):
            X_tr, X_te = X.iloc[tr_idx], X.iloc[te_idx]
            y_tr, y_te = y[tr_idx], y[te_idx]
            if len(np.unique(y_tr)) < 2:
                aucs.append(np.nan); break
            model = fit_classifier(X_tr, pd.Series(y_tr))
            proba = predict_positive_score(model, X_te)
            y_true_all.append(y_te); proba_all.append(proba)
        else:
            y_true = np.concatenate(y_true_all)
            proba = np.concatenate(proba_all)
            aucs.append(float(roc_auc_score(y_true, proba)) if len(np.unique(y_true)) > 1 else np.nan)
            continue
        # breakした場合
        if len(aucs) < len(ks):
            aucs.extend([np.nan] * (len(ks) - len(aucs)))
    return aucs

mask_H = (fair_groups == "High")
mask_L = (fair_groups == "Low")
auc_high = _in_group_auc(mask_H)
auc_low = _in_group_auc(mask_L)

def _best_k(ks_list, auc_vals):
    arr = np.asarray(auc_vals, dtype=float)
    if np.all(~np.isfinite(arr)):
        return None, np.nan
    maxv = np.nanmax(arr)
    cand = np.array(ks_list)[np.isclose(arr, maxv, rtol=1e-6, atol=1e-12)]
    return int(np.min(cand)), float(maxv)

best_k_high, best_auc_high = _best_k(ks, auc_high)
best_k_low, best_auc_low = _best_k(ks, auc_low)

with open(outpath("BEST_K_BY_GROUP.JSON"), "w", encoding="utf-8") as f:
    json.dump({
        "BEST_K_HIGH": best_k_high,
        "BEST_AUC_HIGH": best_auc_high,
        "BEST_K_LOW": best_k_low,
        "BEST_AUC_LOW": best_auc_low,
    }, f, ensure_ascii=False, indent=2)

plt.figure(figsize=(12, 7))
ax = plt.gca()
ax.plot(ks, auc_high, marker="o", label="MSSQ High")
ax.plot(ks, auc_low, marker="s", label="MSSQ Low")
if best_k_high is not None:
    ax.scatter([best_k_high], [best_auc_high], s=160, zorder=5)
    ax.annotate(f"High max={best_auc_high:.3f} (k={best_k_high})",
                xy=(best_k_high, best_auc_high),
                xytext=(best_k_high, best_auc_high + 0.02),
                ha="center", va="bottom", fontsize=18)
if best_k_low is not None:
    ax.scatter([best_k_low], [best_auc_low], s=160, zorder=5)
    ax.annotate(f"Low max={best_auc_low:.3f} (k={best_k_low})",
                xy=(best_k_low, best_auc_low),
                xytext=(best_k_low, best_auc_low + 0.02),
                ha="center", va="bottom", fontsize=18)

ax.invert_xaxis()
ax.set_xlabel("Number of Features (k)")
ax.set_ylabel("ROC AUC (pooled, in-group LOSO)")
ax.set_title("AUC vs Number of Features by MSSQ Group")
ax.grid(True, alpha=0.4)
ax.legend(loc="best")
plt.tight_layout()
plt.savefig(outpath("AUC_VS_K_BY_GROUP.PNG"), dpi=300)
plt.close()

print(f"[Cell3D] BEST_K_HIGH={best_k_high}, BEST_K_LOW={best_k_low}")


# ===== Cell 4A: ROC-AUC vs k（95%信頼区間付き） =====
# 前提: Cell3D まで実行済み（X_all, y_all, groups, SHAP_FEATURE_RANKING.CSV が存在）

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import roc_auc_score

rank_df = pd.read_csv(outpath("SHAP_FEATURE_RANKING.CSV"), index_col=0, encoding="utf-8-sig")
feature_order = [f for f in rank_df.index if f in X_all.columns]
if not feature_order:
    raise RuntimeError("[Cell4A] SHAPランキング上位の特徴が X_all にありません。")

logo = LeaveOneGroupOut()
ks = list(range(len(feature_order), 0, -1))

records = []
for k in ks:
    feats = feature_order[:k]
    X = X_all[feats].astype(np.float32)
    y = y_all.values
    g = groups.values

    y_true_all, proba_all, subj_all = [], [], []
    for tr_idx, te_idx in logo.split(X, y, g):
        X_tr, X_te = X.iloc[tr_idx], X.iloc[te_idx]
        y_tr, y_te = y[tr_idx], y[te_idx]
        if len(np.unique(y_tr)) < 2:
            raise RuntimeError(f"[Cell4A] k={k}: 学習foldが単一クラスです。")

        model = fit_classifier(X_tr, pd.Series(y_tr))
        proba = predict_positive_score(model, X_te)
        y_true_all.append(y_te)
        proba_all.append(proba)
        subj_all.append(g[te_idx])

    y_pool = np.concatenate(y_true_all)
    s_pool = np.concatenate(proba_all)
    subj_pool = np.concatenate(subj_all)
    auc_obs = float(roc_auc_score(y_pool, s_pool))
    records.append({"k": k, "auc": auc_obs})

    # --- 被験者単位ブートストラップで95% CI ---
    df = pd.DataFrame({"subject": subj_pool, "y_true": y_pool, "y_score": s_pool})
    subj_ids = df["subject"].unique()
    B = 2000
    rng = np.random.default_rng(20251101)
    auc_boot = []
    for _ in range(B):
        sampled = rng.choice(subj_ids, size=len(subj_ids), replace=True)
        df_boot = pd.concat([df[df["subject"] == sid] for sid in sampled], ignore_index=True)
        if df_boot["y_true"].nunique() < 2:
            continue
        auc_boot.append(float(roc_auc_score(df_boot["y_true"], df_boot["y_score"])))
    if len(auc_boot) == 0:
        ci_low = ci_high = float("nan")
    else:
        ci_low = float(np.quantile(auc_boot, 0.025))
        ci_high = float(np.quantile(auc_boot, 0.975))
    records[-1].update({"ci_low": ci_low, "ci_high": ci_high})

df_auc = pd.DataFrame(records)
df_auc.to_csv(outpath("AUC_K_CI.csv"), index=False, encoding="utf-8-sig")
print(f"[Cell4A] 保存 -> {outpath('AUC_K_CI.csv')}")

# --- プロット ---
plt.figure(figsize=(10,6))
plt.errorbar(df_auc["k"], df_auc["auc"],
             yerr=[df_auc["auc"]-df_auc["ci_low"], df_auc["ci_high"]-df_auc["auc"]],
             fmt='o', capsize=4, linewidth=1.2, color="tab:blue")
plt.gca().invert_xaxis()
plt.xlabel("Number of features (k)")
plt.ylabel("ROC AUC (pooled)")
plt.title("ROC AUC vs k with 95% CI")
plt.grid(True, alpha=0.4)
plt.tight_layout()
plt.savefig(outpath("AUC_K_CI.png"), dpi=300)
plt.close()
print(f"[Cell4A] 図を保存 -> {outpath('AUC_K_CI.png')}")


In [ ]:
# ===== Inner LOSO folds builder =====

from typing import List
import pandas as pd

def choose_inner_folds_loso(train_subject_ids: List[str]) -> List[List[str]]:
    """
    外側LOSOで得た “学習側の被験者ID” リストを受け取り、
    1名ずつ検証に回す inner-LOSO のfoldリスト（[[sid1], [sid2], ...]）を返す。
    """
    if not isinstance(train_subject_ids, (list, tuple)):
        raise RuntimeError("[inner folds] train_subject_ids は list/tuple を想定しています。")
    uniq = list(pd.unique(pd.Series([str(sid) for sid in train_subject_ids])))
    if len(uniq) == 0:
        raise RuntimeError("[inner folds] train_subject_ids が空です。")
    uniq_sorted = sorted(uniq, key=lambda x: (len(x), x))
    folds = [[sid] for sid in uniq_sorted]
    print(f"[inner folds] {len(folds)} splits -> val subjects = {', '.join(uniq_sorted)}")
    return folds


In [ ]:
# ===== Cell 5A (FINAL, F1専用): inner-LOSO τ最適化（連結val）→ outer予測・評価 =====
# 前提: Cell1〜4 実行済み（X_all, y_all, groups, SUBJECT_META, BEST_K, choose_inner_folds_loso 等）

import numpy as np
import pandas as pd
import sklearn.metrics as skm
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

# ---------------- 基本チェック ----------------
req = ["X_all","y_all","groups","SUBJECT_META",
       "choose_inner_folds_loso","fit_classifier","predict_positive_score","outpath"]
missing = [v for v in req if v not in globals()]
if missing:
    raise RuntimeError(f"[Cell5A] 未定義の変数/関数があります: {missing}")

if "MSSQ_group" not in SUBJECT_META.columns and "MSSQ_group" not in SUBJECT_META.index.names:
    raise RuntimeError("[Cell5A] SUBJECT_META に MSSQ_group 列（または index）が必要である．")

# ---------------- 設定（等間隔グリッド＋近傍再探索） ----------------
COARSE_STEPS = 1001      # 0.0〜1.0 を等間隔
FINE_STEPS   = 101       # 近傍再探索の細かさ
FINE_MARGIN  = 0.01      # 近傍幅（±0.01）
VERBOSE      = True

# ---------------- 入力整形 ----------------
X_base = X_all.astype(np.float32)
y_base = y_all.astype(int)
g_base = groups.astype(str)

# MSSQ group マッピング（High/Lowに正規化）
if "subject_id" in SUBJECT_META.columns:
    mapper = SUBJECT_META.set_index("subject_id")["MSSQ_group"].astype(str).to_dict()
else:
    mapper = SUBJECT_META["MSSQ_group"].astype(str).to_dict()

fair_groups = g_base.map(mapper).astype(str).str.strip().str.lower().map({"high":"High","low":"Low"})
if fair_groups.isna().any():
    raise RuntimeError(f"[Cell5A] MSSQ_group 未割当ID: {sorted(set(g_base[fair_groups.isna()]))}")

# 特徴選抜（SHAP順の上位K）
rank_df = pd.read_csv(outpath("SHAP_FEATURE_RANKING.CSV"), index_col=0, encoding="utf-8-sig")
feature_order = [f for f in rank_df.index if f in X_base.columns]
feats_k = feature_order[:best_k]
if len(feats_k) < best_k:
    print(f"[Cell5A][WARN] ランキング上位に X に無い特徴が混在 → {len(feats_k)} 列で実行．")
X_k = X_base[feats_k]

if VERBOSE:
    print(f"[DBG] 使用特徴数: {len(feats_k)}  (BEST_K={best_k})")

# ---------------- 指標・評価ユーティリティ（F1専用） ----------------
def _conf_from_preds(y_true: np.ndarray, y_pred: np.ndarray):
    TN, FP, FN, TP = skm.confusion_matrix(y_true, y_pred, labels=[0,1]).ravel()
    return TP, FP, FN, TN

def _f1_from_conf(TP, FP, FN, TN) -> float:
    TP = float(TP); FP = float(FP); FN = float(FN)
    denom = (2*TP + FP + FN)
    return (2*TP / denom) if denom > 0 else 0.0

def _f1_binary(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    TP, FP, FN, TN = _conf_from_preds(y_true, y_pred)
    return _f1_from_conf(TP, FP, FN, TN)

def _grid(l, r, steps):
    l = float(max(0.0, l)); r = float(min(1.0, r))
    if l > r: l, r = r, l
    return np.linspace(l, r, int(steps), dtype=float)

# ---------------- τ最適化（Single / WG-F1 / Group-F1） ----------------
def _single_tau_opt(scores: np.ndarray, y: np.ndarray):
    # coarse
    cands = _grid(0.0, 1.0, COARSE_STEPS)
    f1_vec = []
    for t in cands:
        yhat = (scores >= t).astype(int)
        f1_vec.append(_f1_binary(y, yhat))
    f1_vec = np.asarray(f1_vec)
    idx = int(np.nanargmax(f1_vec)); tau0 = float(cands[idx]); best0 = float(f1_vec[idx])

    # fine around tau0
    left  = max(0.0, tau0 - FINE_MARGIN)
    right = min(1.0, tau0 + FINE_MARGIN)
    cands2 = _grid(left, right, FINE_STEPS)
    f1_vec2 = []
    for t in cands2:
        yhat = (scores >= t).astype(int)
        f1_vec2.append(_f1_binary(y, yhat))
    f1_vec2 = np.asarray(f1_vec2)
    idx2 = int(np.nanargmax(f1_vec2)); tau = float(cands2[idx2]); best = float(f1_vec2[idx2])

    return {"tau": tau, "F1_val": best, "tau_coarse": tau0, "F1_coarse": best0}

def _wg_f1_opt_joint(scores: np.ndarray, y: np.ndarray, grp: np.ndarray):
    """
    Fair-MinF1（最悪群F1の最大化）を 2Dグリッド（τH×τL）で探索する．
    戻り値:
      {"tauH","tauL","F1_H_val","F1_L_val","WG_F1_val","F1_pooled_val"}
    """
    maskH = (grp == "High"); maskL = (grp == "Low")
    if (maskH.sum()==0) or (maskL.sum()==0):
        raise RuntimeError("[WG-F1] 連結valに High/Low の両群が必要（どちらかが0件）")

    sH, yH = scores[maskH], y[maskH]
    sL, yL = scores[maskL], y[maskL]

    candH = _grid(0.0, 1.0, COARSE_STEPS)
    candL = _grid(0.0, 1.0, COARSE_STEPS)

    def _f1_vec(s, yy, cands):
        f1 = np.empty_like(cands)
        for i, t in enumerate(cands):
            yhat = (s >= t).astype(int)
            f1[i] = _f1_binary(yy, yhat)
        return f1

    f1H = _f1_vec(sH, yH, candH)
    f1L = _f1_vec(sL, yL, candL)

    best = {"WG": -np.inf, "pooled": -np.inf, "tH": 0.5, "tL": 0.5, "F1H": 0.0, "F1L": 0.0}
    for i, tH in enumerate(candH):
        wg_row = np.minimum(f1H[i], f1L)
        j = int(np.nanargmax(wg_row))
        WG = float(wg_row[j])

        yhatH = (sH >= tH).astype(int)
        yhatL = (sL >= candL[j]).astype(int)
        F1_pooled = _f1_binary(
            np.concatenate([yH, yL]),
            np.concatenate([yhatH, yhatL])
        )

        cand = {"WG": WG, "pooled": float(F1_pooled),
                "tH": float(tH), "tL": float(candL[j]),
                "F1H": float(f1H[i]), "F1L": float(f1L[j])}

        def _is_better(cur, new):
            if new["WG"] > cur["WG"]: return True
            if new["WG"] < cur["WG"]: return False
            if new["pooled"] > cur["pooled"]: return True
            if new["pooled"] < cur["pooled"]: return False
            if abs(new["tH"]-new["tL"]) < abs(cur["tH"]-cur["tL"]): return True
            if abs(new["tH"]-new["tL"]) > abs(cur["tH"]-cur["tL"]): return False
            if (new["tH"], new["tL"]) < (cur["tH"], cur["tL"]): return True
            return False

        if _is_better(best, cand):
            best = cand

    # fine (box refine)
    lH = max(0.0, best["tH"] - FINE_MARGIN); rH = min(1.0, best["tH"] + FINE_MARGIN)
    lL = max(0.0, best["tL"] - FINE_MARGIN); rL = min(1.0, best["tL"] + FINE_MARGIN)
    candH2 = _grid(lH, rH, FINE_STEPS)
    candL2 = _grid(lL, rL, FINE_STEPS)

    f1H2 = _f1_vec(sH, yH, candH2)
    f1L2 = _f1_vec(sL, yL, candL2)

    best2 = dict(best)
    for i, tH in enumerate(candH2):
        wg_row = np.minimum(f1H2[i], f1L2)
        j = int(np.nanargmax(wg_row))
        WG = float(wg_row[j])

        yhatH = (sH >= tH).astype(int)
        yhatL = (sL >= candL2[j]).astype(int)
        F1_pooled = _f1_binary(
            np.concatenate([yH, yL]),
            np.concatenate([yhatH, yhatL])
        )
        cand = {"WG": WG, "pooled": float(F1_pooled),
                "tH": float(tH), "tL": float(candL2[j]),
                "F1H": float(f1H2[i]), "F1L": float(f1L2[j])}

        if (cand["WG"] > best2["WG"] or
            (cand["WG"] == best2["WG"] and (
                cand["pooled"] > best2["pooled"] or
                (cand["pooled"] == best2["pooled"] and (
                    abs(cand["tH"]-cand["tL"]) < abs(best2["tH"]-best2["tL"]) or
                    (abs(cand["tH"]-cand["tL"]) == abs(best2["tH"]-best2["tL"]) and
                     (cand["tH"], cand["tL"]) < (best2["tH"], best2["tL"]))
                ))
            ))):
            best2 = cand

    return {
        "tauH": best2["tH"], "tauL": best2["tL"],
        "F1_H_val": best2["F1H"], "F1_L_val": best2["F1L"],
        "WG_F1_val": best2["WG"], "F1_pooled_val": best2["pooled"],
    }

def _group_f1_opt(scores: np.ndarray, y: np.ndarray, grp: np.ndarray):
    # 2Dグリッド（τH×τL）で pooled F1 最大化（coarse→fine）
    maskH = (grp == "High"); maskL = (grp == "Low")
    if (maskH.sum()==0) or (maskL.sum()==0):
        raise RuntimeError("[Group-F1] 連結valに High/Low の両群が必要（どちらかが0件）")

    sH, yH = scores[maskH], y[maskH]
    sL, yL = scores[maskL], y[maskL]

    def _pooled_f1_for(tH, tL):
        yhatH = (sH >= tH).astype(int)
        yhatL = (sL >= tL).astype(int)
        y_true = np.concatenate([yH, yL])
        y_pred = np.concatenate([yhatH, yhatL])
        return _f1_binary(y_true, y_pred)

    candH = _grid(0.0, 1.0, COARSE_STEPS)
    candL = _grid(0.0, 1.0, COARSE_STEPS)
    best = {"F1_val": -np.inf, "tauH": None, "tauL": None}
    for tH in candH:
        f1_vec = np.empty_like(candL)
        for j, tL in enumerate(candL):
            f1_vec[j] = _pooled_f1_for(tH, tL)
        jmax = int(np.nanargmax(f1_vec))
        if float(f1_vec[jmax]) > best["F1_val"]:
            best.update({"F1_val": float(f1_vec[jmax]), "tauH": float(tH), "tauL": float(candL[jmax])})

    # fine
    lH = max(0.0, best["tauH"] - FINE_MARGIN); rH = min(1.0, best["tauH"] + FINE_MARGIN)
    lL = max(0.0, best["tauL"] - FINE_MARGIN); rL = min(1.0, best["tauL"] + FINE_MARGIN)
    candH2 = _grid(lH, rH, FINE_STEPS); candL2 = _grid(lL, rL, FINE_STEPS)

    best2 = dict(best)
    for tH in candH2:
        f1_vec2 = np.empty_like(candL2)
        for j, tL in enumerate(candL2):
            f1_vec2[j] = _pooled_f1_for(tH, tL)
        jmax = int(np.nanargmax(f1_vec2))
        if float(f1_vec2[jmax]) > best2["F1_val"]:
            best2.update({"F1_val": float(f1_vec2[jmax]), "tauH": float(tH), "tauL": float(candL2[jmax])})

    return best2  # {"tauH","tauL","F1_val"}

# ---------------- outer LOSO with inner concatenation ----------------
logo_outer = LeaveOneGroupOut()
rows, pred_rows = [], []

for fold_id, (tr_idx, te_idx) in enumerate(logo_outer.split(X_k, y_base.values, g_base.values), start=1):
    train_mask = pd.Series(False, index=g_base.index); train_mask.iloc[tr_idx] = True
    test_mask  = pd.Series(False, index=g_base.index);  test_mask.iloc[te_idx]  = True
    test_sid = g_base.iloc[te_idx].iloc[0]

    # ===== inner folds =====
    inner_ids   = sorted(g_base[train_mask].unique())
    inner_folds = choose_inner_folds_loso(inner_ids)
    if VERBOSE:
        print(f"[inner folds] {len(inner_folds)} splits -> val subjects = {', '.join(inner_ids)}")

    # 連結valの器
    val_scores_all, val_y_all, val_grp_all = [], [], []

    # --- inner train に両群が居るか（群別τの前提） ---
    inner_train_groups = fair_groups[train_mask]
    if not (("High" in set(inner_train_groups)) and ("Low" in set(inner_train_groups))):
        raise RuntimeError(f"[Cell5A] fold{fold_id}: inner-train に両群(High/Low)が必要（群別τの前提）")

    # ===== inner: train→val 予測の連結 =====
    for inner_val in inner_folds:
        val_mask  = g_base.isin(inner_val) & train_mask
        trn_mask  = train_mask & (~val_mask)
        if not trn_mask.any() or not val_mask.any():
            continue

        X_tr, y_tr = X_k[trn_mask], y_base[trn_mask]
        X_vl, y_vl = X_k[val_mask], y_base[val_mask]
        grp_vl     = fair_groups[val_mask].to_numpy()

        # inner 学習
        model_inner = fit_classifier(X_tr, y_tr)
        sc_vl = predict_positive_score(model_inner, X_vl).astype(float)

        # 連結（生の val 予測）
        val_scores_all.append(sc_vl)
        val_y_all.append(y_vl.to_numpy())
        val_grp_all.append(grp_vl)

        if VERBOSE:
            nH = int((grp_vl=="High").sum()); nL = int((grp_vl=="Low").sum())
            print(f"[DBG] fold{fold_id}: inner_val size={len(y_vl):3d}  High={nH} Low={nL}")

    # 連結
    if len(val_scores_all) == 0:
        raise RuntimeError(f"[Cell5A] fold{fold_id}: inner validation が空である．")
    s_val = np.concatenate(val_scores_all)
    y_val = np.concatenate(val_y_all)
    g_val = np.concatenate(val_grp_all)

    # ===== 連結valで一度だけ τ を最適化（F1） =====
    res_single = _single_tau_opt(s_val, y_val)
    tau_single = float(res_single["tau"])

    res_wg = _wg_f1_opt_joint(s_val, y_val, g_val)
    tauH_wg, tauL_wg = float(res_wg["tauH"]), float(res_wg["tauL"])

    res_group = _group_f1_opt(s_val, y_val, g_val)
    tauH_grp, tauL_grp = float(res_group["tauH"]), float(res_group["tauL"])

    if VERBOSE:
        print(f"[DBG] fold{fold_id}: tau_single={tau_single:.6f} | "
              f"Group-F1(tH,tL)=({tauH_grp:.6f},{tauL_grp:.6f}) | "
              f"WG-F1(tH,tL)=({tauH_wg:.6f},{tauL_wg:.6f})")

    # ===== outer test =====
    X_tr_o, y_tr_o = X_k[train_mask], y_base[train_mask]
    X_te_o, y_te_o = X_k[test_mask],  y_base[test_mask]
    grp_te         = fair_groups[test_mask].to_numpy()

    model_outer = fit_classifier(X_tr_o, y_tr_o)
    sc_te = predict_positive_score(model_outer, X_te_o).astype(float)

    yhat_single = (sc_te >= tau_single).astype(int)
    yhat_groupF1 = (sc_te >= np.where(grp_te=="High", tauH_grp, tauL_grp)).astype(int)
    yhat_wgF1    = (sc_te >= np.where(grp_te=="High", tauH_wg,  tauL_wg )).astype(int)

    F1_single = _f1_binary(y_te_o.to_numpy(), yhat_single)
    F1_group  = _f1_binary(y_te_o.to_numpy(), yhat_groupF1)
    F1_wg     = _f1_binary(y_te_o.to_numpy(), yhat_wgF1)

    if VERBOSE:
        same_grp = bool(np.array_equal(yhat_single, yhat_groupF1))
        same_wg  = bool(np.array_equal(yhat_single, yhat_wgF1))
        nH_te = int((grp_te=="High").sum()); nL_te = int((grp_te=="Low").sum())
        print(f"[DBG] fold{fold_id}: TEST High={nH_te} Low={nL_te} | "
              f"Single==Group-F1: {same_grp}  Single==WG-F1: {same_wg} | "
              f"F1(single, group, wg)=({F1_single:.3f}, {F1_group:.3f}, {F1_wg:.3f})")

    rows.append({
        "fold_id": int(fold_id),
        "test_id": str(test_sid),
        "best_k": int(best_k),
        "tau_single": float(tau_single),
        "tau_high_GroupF1": float(tauH_grp), "tau_low_GroupF1": float(tauL_grp),
        "tau_high_WGF1": float(tauH_wg),     "tau_low_WGF1": float(tauL_wg),
        "F1_single": float(F1_single),
        "F1_group":  float(F1_group),
        "F1_group_WG": float(F1_wg),
        "n_test": int(len(y_te_o)),
    })

    # 予測詳細（F1専用命名）
    for yy, ss, gg, ys, yg, yw in zip(y_te_o, sc_te, grp_te, yhat_single, yhat_groupF1, yhat_wgF1):
        pred_rows.append({
            "fold_id": int(fold_id),
            "test_id": str(test_sid),
            "y_true": int(yy),
            "proba": float(ss),
            "group": str(gg),
            "y_pred_single": int(ys),
            "y_pred_group_F1": int(yg),
            "y_pred_group_WG": int(yw),
        })
    
    print(f"[DBG] fold{fold_id}: WG-joint val -> "
          f"F1_H={res_wg['F1_H_val']:.3f}, F1_L={res_wg['F1_L_val']:.3f}, "
          f"WG(F1)={res_wg['WG_F1_val']:.3f}, pooled(F1)={res_wg['F1_pooled_val']:.3f}, "
          f"(tH,tL)=({tauH_wg:.4f},{tauL_wg:.4f})")

# ---------------- 出力 ----------------
df_fold = pd.DataFrame(rows)
df_pred = pd.DataFrame(pred_rows)
df_fold.to_csv(outpath("GROUP_AWARE_THRESH_BY_FOLD.CSV"), index=False, encoding="utf-8-sig")
df_pred.to_csv(outpath("GROUP_AWARE_PREDICTIONS.CSV"), index=False, encoding="utf-8-sig")

# 参考: プールAUC（確率は共通なので1つ）
auc_pool = float(roc_auc_score(df_pred["y_true"].to_numpy(), df_pred["proba"].to_numpy()))

def _pooled_f1(col):
    y_true = df_pred["y_true"].to_numpy()
    yhat   = df_pred[col].to_numpy().astype(int)
    return _f1_binary(y_true, yhat)

summary = {
    "best_k": int(best_k),
    "AUC_pooled": auc_pool,  # 不要ならこのキーごと削除可
    "F1_pooled_single": _pooled_f1("y_pred_single"),
    "F1_pooled_group":  _pooled_f1("y_pred_group_F1"),
    "F1_pooled_group_WG": _pooled_f1("y_pred_group_WG"),
    "metric": "F1",
    "n_samples": int(len(df_pred)),
    "n_pos": int((df_pred["y_true"]==1).sum()),
    "n_neg": int((df_pred["y_true"]==0).sum()),
}
pd.DataFrame([summary]).to_csv(outpath("GROUP_AWARE_SUMMARY.CSV"), index=False, encoding="utf-8-sig")
print(f"[Cell5A] Done: outer folds={len(df_fold)}, pooled AUC={auc_pool:.3f}")
print(f"[Cell5A] Summary: {summary}")

# ---------------- 混同行列プロット（F1専用ラベル） ----------------
def _draw_cm(cm, title, path):
    fig, ax = plt.subplots(figsize=(7,6))
    im = ax.imshow(cm, cmap="Blues", vmin=0, vmax=max(cm.max(),1))
    labels = np.array([["TN","FP"],["FN","TP"]])
    for i in range(2):
        for j in range(2):
            val = int(cm[i,j])
            color = "white" if val > 0.6*im.get_clim()[1] else "black"
            ax.text(j, i, f"{labels[i,j]}\n{val}", ha="center", va="center",
                    fontsize=18, fontweight="bold", color=color)
    ax.set_xticks([0,1]); ax.set_xticklabels(["Pred:0","Pred:1"])
    ax.set_yticks([0,1]); ax.set_yticklabels(["True:0","True:1"], rotation=90, va="center")
    ax.set_title(title); ax.grid(False)
    plt.tight_layout(); plt.savefig(path, dpi=300); plt.close()

y_pool = df_pred["y_true"].to_numpy()
cm_single = skm.confusion_matrix(y_pool, df_pred["y_pred_single"],    labels=[0,1])
cm_group  = skm.confusion_matrix(y_pool, df_pred["y_pred_group_F1"],  labels=[0,1])
cm_wg     = skm.confusion_matrix(y_pool, df_pred["y_pred_group_WG"],  labels=[0,1])

_draw_cm(cm_single, f"Cell5A Single τ (F1={summary['F1_pooled_single']:.3f})",
         outpath("CONFMAT_CELL5A_SINGLE(F1).png"))
_draw_cm(cm_group,  f"Cell5A Group-F1 τ (F1={summary['F1_pooled_group']:.3f})",
         outpath("CONFMAT_CELL5A_GROUP(F1).png"))
_draw_cm(cm_wg,     f"Cell5A WG-F1 τ (F1={summary['F1_pooled_group_WG']:.3f})",
         outpath("CONFMAT_CELL5A_WG(F1).png"))
print("[Cell5A] Confusion matrices saved -> CONFMAT_CELL5A_*.png")


In [ ]:
# ===== Cell 5B++: 確率スコア分布（OVERALL/群別/各Fold）＋ τ の中央値/IQR 帯・±表記 =====
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ---------- 設定 ----------
BINS = 40
LW = 1.5
FS_TITLE, FS_LABEL, FS_LEGEND, FS_TICK = 30, 24, 20, 20

# 色指定（ユーザ指定）
COLOR_SICK = "red"   # True:Sick
COLOR_NON  = "blue"  # True:Non-Sick

# 線色（しきい値）
COLOR_SINGLE = "black"
COLOR_GROUP  = "green"
COLOR_WG     = "purple"

# 出力フォルダ（新規作成）
RUN_ROOT = os.path.dirname(outpath("__dummy__"))
IMG_DIR  = os.path.join(RUN_ROOT, "PROBA_DIST_F1")
os.makedirs(IMG_DIR, exist_ok=True)

# ファイルパス
SAVE_OVERALL_SvG   = os.path.join(IMG_DIR, "OVERALL_SvGroup.png")
SAVE_OVERALL_SvWG  = os.path.join(IMG_DIR, "OVERALL_SvWG.png")
SAVE_BYGROUP_SvG   = os.path.join(IMG_DIR, "BYGROUP_SvGroup.png")
SAVE_BYGROUP_SvWG  = os.path.join(IMG_DIR, "BYGROUP_SvWG.png")

# ---------- 入力 ----------
pred_path = outpath("GROUP_AWARE_PREDICTIONS.CSV")
fold_path = outpath("GROUP_AWARE_THRESH_BY_FOLD.CSV")
if not (os.path.exists(pred_path) and os.path.exists(fold_path)):
    raise FileNotFoundError("[Cell5B++] 必要CSVが見つからない（Cell 5A を先に実行）")

df_pred = pd.read_csv(pred_path, encoding="utf-8-sig")
df_fold = pd.read_csv(fold_path, encoding="utf-8-sig")

# ---------- モード自動判定（F1 or BA） ----------
cols_f1 = {"high":"tau_high_GroupF1", "low":"tau_low_GroupF1", "wgh":"tau_high_WGF1", "wgl":"tau_low_WGF1"}
cols_ba = {"high":"tau_high_GroupBA", "low":"tau_low_GroupBA", "wgh":"tau_high_WGBA", "wgl":"tau_low_WGBA"}

if all(c in df_fold.columns for c in [cols_f1["high"], cols_f1["low"]]):
    mode = "F1"
    c_high, c_low, c_wgh, c_wgl = cols_f1["high"], cols_f1["low"], cols_f1["wgh"], cols_f1["wgl"]
    pred_group_col = "y_pred_group_F1"
elif all(c in df_fold.columns for c in [cols_ba["high"], cols_ba["low"]]):
    mode = "BA"
    c_high, c_low, c_wgh, c_wgl = cols_ba["high"], cols_ba["low"], cols_ba["wgh"], cols_ba["wgl"]
    pred_group_col = "y_pred_group_BA"
else:
    raise RuntimeError("[Cell5B++] しきい値列が見つからない（F1/BAどちらかのCell 5Aの出力が必要）")

# ---------- 集約: 中央値/IQR（Q1〜Q3） ----------
def _qstats(s):
    s = pd.to_numeric(s, errors="coerce")
    s = s[np.isfinite(s)]
    if s.size == 0:
        return np.nan, np.nan, np.nan, np.nan  # med, q1, q3, half_iqr
    med = float(np.nanmedian(s))
    q1, q3 = np.nanpercentile(s, [25, 75])
    half = float((q3 - q1)/2.0)
    return float(med), float(q1), float(q3), half

tau_single_med, tau_single_q1, tau_single_q3, tau_single_half = _qstats(df_fold["tau_single"])
tau_high_med,   tau_high_q1,   tau_high_q3,   tau_high_half   = _qstats(df_fold[c_high])
tau_low_med,    tau_low_q1,    tau_low_q3,    tau_low_half    = _qstats(df_fold[c_low])
tau_high_wg,    tau_high_wg_q1, tau_high_wg_q3, tau_high_wg_half = _qstats(df_fold[c_wgh]) if c_wgh in df_fold.columns else (np.nan, np.nan, np.nan, np.nan)
tau_low_wg,     tau_low_wg_q1,  tau_low_wg_q3,  tau_low_wg_half  = _qstats(df_fold[c_wgl]) if c_wgl in df_fold.columns else (np.nan, np.nan, np.nan, np.nan)

# ---------- データ分解 ----------
proba = pd.to_numeric(df_pred["proba"], errors="coerce").values
ytrue = pd.to_numeric(df_pred["y_true"], errors="coerce").values.astype(int)
grp   = df_pred["group"].astype(str).str.strip()

p_sick = proba[ytrue == 1]   # True:Sick
p_non  = proba[ytrue == 0]   # True:Non-Sick
n_sick, n_non = len(p_sick), len(p_non)

maskH = (grp == "High"); maskL = (grp == "Low")

# ---------- ユーティリティ ----------
def _style_axes(ax, title=None):
    if title: ax.set_title(title, fontsize=FS_TITLE)
    ax.set_xlabel("Predicted probability", fontsize=FS_LABEL)
    ax.set_ylabel("Density", fontsize=FS_LABEL)
    ax.tick_params(axis="both", labelsize=FS_TICK)
    ax.set_xlim(0, 1)

def _hist_overall(ax):
    ax.hist(p_sick, bins=BINS, density=True, alpha=0.5, label=f"True:Sick (n={n_sick})", color=COLOR_SICK)
    ax.hist(p_non,  bins=BINS, density=True, alpha=0.5, label=f"True:Non-Sick (n={n_non})", color=COLOR_NON)

def _hist_bygroup(axes):
    # High
    p_sick_H = proba[(ytrue==1) & maskH]; p_non_H = proba[(ytrue==0) & maskH]
    axes[0].hist(p_sick_H, bins=BINS, density=True, alpha=0.5, label=f"True:Sick (n={len(p_sick_H)})", color=COLOR_SICK)
    axes[0].hist(p_non_H,  bins=BINS, density=True, alpha=0.5, label=f"True:Non-Sick (n={len(p_non_H)})", color=COLOR_NON)
    _style_axes(axes[0], "High group")
    # Low
    p_sick_L = proba[(ytrue==1) & maskL]; p_non_L = proba[(ytrue==0) & maskL]
    axes[1].hist(p_sick_L, bins=BINS, density=True, alpha=0.5, label=f"True:Sick (n={len(p_sick_L)})", color=COLOR_SICK)
    axes[1].hist(p_non_L,  bins=BINS, density=True, alpha=0.5, label=f"True:Non-Sick (n={len(p_non_L)})", color=COLOR_NON)
    _style_axes(axes[1], "Low group")

def _vline_with_iqr(ax, x_med, q1, q3, color, ls, label_core):
    if np.isfinite(x_med):
        ax.axvline(x_med, color=color, linestyle=ls, linewidth=LW,
                   label=f"{label_core} = {x_med:.3f} ± {(q3-q1)/2:.3f}" if (np.isfinite(q1) and np.isfinite(q3)) else f"{label_core} = {x_med:.3f}")
    if np.isfinite(q1) and np.isfinite(q3):
        ax.axvspan(q1, q3, color=color, alpha=0.12)

# ---------- 1) OVERALL: Single vs Group（中央値・IQR帯・±表記） ----------
fig, ax = plt.subplots(figsize=(9,6))
_hist_overall(ax)
_vline_with_iqr(ax, tau_single_med, tau_single_q1, tau_single_q3, COLOR_SINGLE, "-", "τ_single")
_vline_with_iqr(ax, tau_high_med,   tau_high_q1,   tau_high_q3,   COLOR_GROUP,  "--", f"τ_high_{mode}")
_vline_with_iqr(ax, tau_low_med,    tau_low_q1,    tau_low_q3,    COLOR_GROUP,  "--", f"τ_low_{mode}")
_style_axes(ax, title=f"Probability distribution (OVERALL) — Single vs Group [{mode}]")
ax.legend(fontsize=FS_LEGEND)
plt.tight_layout(); plt.savefig(SAVE_OVERALL_SvG, dpi=300); plt.close()
print(f"[Cell5B++] Saved -> {SAVE_OVERALL_SvG}")

# ---------- 2) OVERALL: Single vs WG（中央値・IQR帯・±表記） ----------
fig, ax = plt.subplots(figsize=(9,6))
_hist_overall(ax)
_vline_with_iqr(ax, tau_single_med, tau_single_q1, tau_single_q3, COLOR_SINGLE, "-", "τ_single")
_vline_with_iqr(ax, tau_high_wg,    tau_high_wg_q1, tau_high_wg_q3, COLOR_WG, ":", "τ_high_WG")
_vline_with_iqr(ax, tau_low_wg,     tau_low_wg_q1,  tau_low_wg_q3,  COLOR_WG, ":", "τ_low_WG")
_style_axes(ax, title=f"Probability distribution (OVERALL) — Single vs WG [{mode}]")
ax.legend(fontsize=FS_LEGEND)
plt.tight_layout(); plt.savefig(SAVE_OVERALL_SvWG, dpi=300); plt.close()
print(f"[Cell5B++] Saved -> {SAVE_OVERALL_SvWG}")

# ---------- 3) BY_GROUP: Single vs Group（各群パネルに該当の IQR帯） ----------
fig, axes = plt.subplots(2, 1, figsize=(9,10), sharex=True)
_hist_bygroup(axes)
# Single は両段に表示
_vline_with_iqr(axes[0], tau_single_med, tau_single_q1, tau_single_q3, COLOR_SINGLE, "-", "τ_single")
_vline_with_iqr(axes[1], tau_single_med, tau_single_q1, tau_single_q3, COLOR_SINGLE, "-", "τ_single")
# High 段に High の Group τ、Low 段に Low の Group τ
_vline_with_iqr(axes[0], tau_high_med, tau_high_q1, tau_high_q3, COLOR_GROUP, "--", f"τ_high_{mode}")
_vline_with_iqr(axes[1], tau_low_med,  tau_low_q1,  tau_low_q3,  COLOR_GROUP, "--", f"τ_low_{mode}")
axes[0].legend(fontsize=FS_LEGEND); axes[1].legend(fontsize=FS_LEGEND)
plt.tight_layout(); plt.savefig(SAVE_BYGROUP_SvG, dpi=300); plt.close()
print(f"[Cell5B++] Saved -> {SAVE_BYGROUP_SvG}")

# ---------- 4) BY_GROUP: Single vs WG（各群パネルに該当の IQR帯） ----------
fig, axes = plt.subplots(2, 1, figsize=(9,10), sharex=True)
_hist_bygroup(axes)
_vline_with_iqr(axes[0], tau_single_med, tau_single_q1, tau_single_q3, COLOR_SINGLE, "-", "τ_single")
_vline_with_iqr(axes[1], tau_single_med, tau_single_q1, tau_single_q3, COLOR_SINGLE, "-", "τ_single")
_vline_with_iqr(axes[0], tau_high_wg, tau_high_wg_q1, tau_high_wg_q3, COLOR_WG, ":", "τ_high_WG")
_vline_with_iqr(axes[1], tau_low_wg,  tau_low_wg_q1,  tau_low_wg_q3,  COLOR_WG, ":", "τ_low_WG")
axes[0].legend(fontsize=FS_LEGEND); axes[1].legend(fontsize=FS_LEGEND)
plt.tight_layout(); plt.savefig(SAVE_BYGROUP_SvWG, dpi=300); plt.close()
print(f"[Cell5B++] Saved -> {SAVE_BYGROUP_SvWG}")

# ---------- 5) Fold単位：OVERALL の確率分布と各Foldの τ（Single vs Group / Single vs WG） ----------
#   - 各Foldのテストサンプルのみでヒストを作図
#   - しきい値は「そのFold行」の値を直接表示（±は不要／IQR帯は使わない）
for _, row in df_fold.iterrows():
    fid = int(row["fold_id"]) if "fold_id" in row else None
    test_id = str(row.get("test_id", f"fold{fid}"))
    sub = df_pred[df_pred["fold_id"] == fid] if "fold_id" in df_pred.columns and fid is not None else df_pred.copy()

    p = pd.to_numeric(sub["proba"], errors="coerce").values
    yt = pd.to_numeric(sub["y_true"], errors="coerce").values.astype(int)
    p_s, p_n = p[yt==1], p[yt==0]

    # 値（このFoldの τ）
    t_single = float(row["tau_single"])
    t_high   = float(row[c_high]) if c_high in row else np.nan
    t_low    = float(row[c_low])  if c_low  in row else np.nan
    t_high_w = float(row[c_wgh])  if c_wgh  in row else np.nan
    t_low_w  = float(row[c_wgl])  if c_wgl  in row else np.nan

    # 出力パス
    p_sg  = os.path.join(IMG_DIR, f"FOLD{fid:02d}_{test_id}_OVERALL_SvGroup.png")
    p_wg  = os.path.join(IMG_DIR, f"FOLD{fid:02d}_{test_id}_OVERALL_SvWG.png")

    # Single vs Group
    fig, ax = plt.subplots(figsize=(9,6))
    ax.hist(p_s, bins=BINS, density=True, alpha=0.5, label=f"True:Sick (n={len(p_s)})", color=COLOR_SICK)
    ax.hist(p_n, bins=BINS, density=True, alpha=0.5, label=f"True:Non-Sick (n={len(p_n)})", color=COLOR_NON)
    if np.isfinite(t_single): ax.axvline(t_single, color=COLOR_SINGLE, linestyle="-", linewidth=LW, label=f"τ_single = {t_single:.3f}")
    if np.isfinite(t_high):   ax.axvline(t_high,   color=COLOR_GROUP,  linestyle="--", linewidth=LW, label=f"τ_high_{mode} = {t_high:.3f}")
    if np.isfinite(t_low):    ax.axvline(t_low,    color=COLOR_GROUP,  linestyle="--", linewidth=LW, label=f"τ_low_{mode}  = {t_low:.3f}")
    _style_axes(ax, title=f"[Fold {fid}] OVERALL — Single vs Group [{mode}]  (test={test_id})")
    ax.legend(fontsize=FS_LEGEND)
    plt.tight_layout(); plt.savefig(p_sg, dpi=300); plt.close()
    print(f"[Cell5B++] Saved -> {p_sg}")

    # Single vs WG
    fig, ax = plt.subplots(figsize=(9,6))
    ax.hist(p_s, bins=BINS, density=True, alpha=0.5, label=f"True:Sick (n={len(p_s)})", color=COLOR_SICK)
    ax.hist(p_n, bins=BINS, density=True, alpha=0.5, label=f"True:Non-Sick (n={len(p_n)})", color=COLOR_NON)
    if np.isfinite(t_single): ax.axvline(t_single, color=COLOR_SINGLE, linestyle="-", linewidth=LW, label=f"τ_single = {t_single:.3f}")
    if np.isfinite(t_high_w): ax.axvline(t_high_w, color=COLOR_WG,     linestyle=":", linewidth=LW, label=f"τ_high_WG = {t_high_w:.3f}")
    if np.isfinite(t_low_w):  ax.axvline(t_low_w,  color=COLOR_WG,     linestyle=":", linewidth=LW, label=f"τ_low_WG  = {t_low_w:.3f}")
    _style_axes(ax, title=f"[Fold {fid}] OVERALL — Single vs WG [{mode}]  (test={test_id})")
    ax.legend(fontsize=FS_LEGEND)
    plt.tight_layout(); plt.savefig(p_wg, dpi=300); plt.close()
    print(f"[Cell5B++] Saved -> {p_wg}")

print(f"[Cell5B++] All images saved in: {IMG_DIR}")


In [ ]:
# ===== Cell 5A (FINAL, BA専用): inner-LOSO τ最適化（連結val）→ outer予測・評価 =====
# 前提: Cell1〜4 実行済み（X_all, y_all, groups, SUBJECT_META, BEST_K, choose_inner_folds_loso 等）

import numpy as np
import pandas as pd
import sklearn.metrics as skm
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

# ---------------- 基本チェック ----------------
req = ["X_all","y_all","groups","SUBJECT_META",
       "choose_inner_folds_loso","fit_classifier","predict_positive_score","outpath"]
missing = [v for v in req if v not in globals()]
if missing:
    raise RuntimeError(f("[Cell5A] 未定義の変数/関数があります: {missing}"))

if "MSSQ_group" not in SUBJECT_META.columns and "MSSQ_group" not in SUBJECT_META.index.names:
    raise RuntimeError("[Cell5A] SUBJECT_META に MSSQ_group 列（または index）が必要である．")

# ---------------- 設定（等間隔グリッド＋近傍再探索） ----------------
COARSE_STEPS = 1001      # 0.0〜1.0 を等間隔
FINE_STEPS   = 101       # 近傍再探索の細かさ
FINE_MARGIN  = 0.01      # 近傍幅（±0.01）
VERBOSE      = True

# ---------------- 入力整形 ----------------
X_base = X_all.astype(np.float32)
y_base = y_all.astype(int)
g_base = groups.astype(str)

# MSSQ group マッピング（High/Lowに正規化）
if "subject_id" in SUBJECT_META.columns:
    mapper = SUBJECT_META.set_index("subject_id")["MSSQ_group"].astype(str).to_dict()
else:
    mapper = SUBJECT_META["MSSQ_group"].astype(str).to_dict()

fair_groups = g_base.map(mapper).astype(str).str.strip().str.lower().map({"high":"High","low":"Low"})
if fair_groups.isna().any():
    raise RuntimeError(f"[Cell5A] MSSQ_group 未割当ID: {sorted(set(g_base[fair_groups.isna()]))}")

# 特徴選抜（SHAP順の上位K）
rank_df = pd.read_csv(outpath("SHAP_FEATURE_RANKING.CSV"), index_col=0, encoding="utf-8-sig")
feature_order = [f for f in rank_df.index if f in X_base.columns]
feats_k = feature_order[:best_k]
if len(feats_k) < best_k:
    print(f"[Cell5A][WARN] ランキング上位に X に無い特徴が混在 → {len(feats_k)} 列で実行．")
X_k = X_base[feats_k]

if VERBOSE:
    print(f"[DBG] 使用特徴数: {len(feats_k)}  (BEST_K={best_k})")

# ---------------- 指標・評価ユーティリティ（BA専用） ----------------
def _conf_from_preds(y_true: np.ndarray, y_pred: np.ndarray):
    TN, FP, FN, TP = skm.confusion_matrix(y_true, y_pred, labels=[0,1]).ravel()
    return TP, FP, FN, TN

def _ba_from_conf(TP, FP, FN, TN) -> float:
    TP = float(TP); FP = float(FP); FN = float(FN); TN = float(TN)
    # Sensitivity/TPR と Specificity/TNR
    tpr = TP / (TP + FN) if (TP + FN) > 0 else 0.0
    tnr = TN / (TN + FP) if (TN + FP) > 0 else 0.0
    return 0.5 * (tpr + tnr)

def _ba_binary(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    TP, FP, FN, TN = _conf_from_preds(y_true, y_pred)
    return _ba_from_conf(TP, FP, FN, TN)

def _grid(l, r, steps):
    l = float(max(0.0, l)); r = float(min(1.0, r))
    if l > r: l, r = r, l
    return np.linspace(l, r, int(steps), dtype=float)

# ---------------- τ最適化（Single / WG-BA / Group-BA） ----------------
def _single_tau_opt(scores: np.ndarray, y: np.ndarray):
    # coarse
    cands = _grid(0.0, 1.0, COARSE_STEPS)
    ba_vec = []
    for t in cands:
        yhat = (scores >= t).astype(int)
        ba_vec.append(_ba_binary(y, yhat))
    ba_vec = np.asarray(ba_vec)
    idx = int(np.nanargmax(ba_vec)); tau0 = float(cands[idx]); best0 = float(ba_vec[idx])

    # fine around tau0
    left  = max(0.0, tau0 - FINE_MARGIN)
    right = min(1.0, tau0 + FINE_MARGIN)
    cands2 = _grid(left, right, FINE_STEPS)
    ba_vec2 = []
    for t in cands2:
        yhat = (scores >= t).astype(int)
        ba_vec2.append(_ba_binary(y, yhat))
    ba_vec2 = np.asarray(ba_vec2)
    idx2 = int(np.nanargmax(ba_vec2)); tau = float(cands2[idx2]); best = float(ba_vec2[idx2])

    return {"tau": tau, "BA_val": best, "tau_coarse": tau0, "BA_coarse": best0}

def _wg_ba_opt_joint(scores: np.ndarray, y: np.ndarray, grp: np.ndarray):
    """
    Fair-MinBA（最悪群BAの最大化）を 2Dグリッド（τH×τL）で探索する．
    戻り値: {"tauH","tauL","BA_H_val","BA_L_val","WG_BA_val","BA_pooled_val"}
    """
    maskH = (grp == "High"); maskL = (grp == "Low")
    if (maskH.sum()==0) or (maskL.sum()==0):
        raise RuntimeError("[WG-BA] 連結valに High/Low の両群が必要（どちらかが0件）")

    sH, yH = scores[maskH], y[maskH]
    sL, yL = scores[maskL], y[maskL]

    candH = _grid(0.0, 1.0, COARSE_STEPS)
    candL = _grid(0.0, 1.0, COARSE_STEPS)

    def _ba_vec(s, yy, cands):
        ba = np.empty_like(cands)
        for i, t in enumerate(cands):
            yhat = (s >= t).astype(int)
            ba[i] = _ba_binary(yy, yhat)
        return ba

    baH = _ba_vec(sH, yH, candH)
    baL = _ba_vec(sL, yL, candL)

    best = {"WG": -np.inf, "pooled": -np.inf, "tH": 0.5, "tL": 0.5, "BAH": 0.0, "BAL": 0.0}
    for i, tH in enumerate(candH):
        wg_row = np.minimum(baH[i], baL)
        j = int(np.nanargmax(wg_row))
        WG = float(wg_row[j])

        yhatH = (sH >= tH).astype(int)
        yhatL = (sL >= candL[j]).astype(int)
        BA_pooled = _ba_binary(
            np.concatenate([yH, yL]),
            np.concatenate([yhatH, yhatL])
        )

        cand = {"WG": WG, "pooled": float(BA_pooled),
                "tH": float(tH), "tL": float(candL[j]),
                "BAH": float(baH[i]), "BAL": float(baL[j])}

        # tie-break: pooled BA → |τH-τL| 小 → (τH,τL) 辞書順小
        def _is_better(cur, new):
            if new["WG"] > cur["WG"]: return True
            if new["WG"] < cur["WG"]: return False
            if new["pooled"] > cur["pooled"]: return True
            if new["pooled"] < cur["pooled"]: return False
            if abs(new["tH"]-new["tL"]) < abs(cur["tH"]-cur["tL"]): return True
            if abs(new["tH"]-new["tL"]) > abs(cur["tH"]-cur["tL"]): return False
            if (new["tH"], new["tL"]) < (cur["tH"], cur["tL"]): return True
            return False

        if _is_better(best, cand):
            best = cand

    # fine (box refine)
    lH = max(0.0, best["tH"] - FINE_MARGIN); rH = min(1.0, best["tH"] + FINE_MARGIN)
    lL = max(0.0, best["tL"] - FINE_MARGIN); rL = min(1.0, best["tL"] + FINE_MARGIN)
    candH2 = _grid(lH, rH, FINE_STEPS)
    candL2 = _grid(lL, rL, FINE_STEPS)

    baH2 = _ba_vec(sH, yH, candH2)
    baL2 = _ba_vec(sL, yL, candL2)

    best2 = dict(best)
    for i, tH in enumerate(candH2):
        wg_row = np.minimum(baH2[i], baL2)
        j = int(np.nanargmax(wg_row))
        WG = float(wg_row[j])

        yhatH = (sH >= tH).astype(int)
        yhatL = (sL >= candL2[j]).astype(int)
        BA_pooled = _ba_binary(
            np.concatenate([yH, yL]),
            np.concatenate([yhatH, yhatL])
        )
        cand = {"WG": WG, "pooled": float(BA_pooled),
                "tH": float(tH), "tL": float(candL2[j]),
                "BAH": float(baH2[i]), "BAL": float(baL2[j])}

        if (cand["WG"] > best2["WG"] or
            (cand["WG"] == best2["WG"] and (
                cand["pooled"] > best2["pooled"] or
                (cand["pooled"] == best2["pooled"] and (
                    abs(cand["tH"]-cand["tL"]) < abs(best2["tH"]-best2["tL"]) or
                    (abs(cand["tH"]-cand["tL"]) == abs(best2["tH"]-best2["tL"]) and
                     (cand["tH"], cand["tL"]) < (best2["tH"], best2["tL"]))
                ))
            ))):
            best2 = cand

    return {
        "tauH": best2["tH"], "tauL": best2["tL"],
        "BA_H_val": best2["BAH"], "BA_L_val": best2["BAL"],
        "WG_BA_val": best2["WG"], "BA_pooled_val": best2["pooled"],
    }

def _group_ba_opt(scores: np.ndarray, y: np.ndarray, grp: np.ndarray):
    # 2Dグリッド（τH×τL）で pooled BA 最大化（coarse→fine）
    maskH = (grp == "High"); maskL = (grp == "Low")
    if (maskH.sum()==0) or (maskL.sum()==0):
        raise RuntimeError("[Group-BA] 連結valに High/Low の両群が必要（どちらかが0件）")

    sH, yH = scores[maskH], y[maskH]
    sL, yL = scores[maskL], y[maskL]

    def _pooled_ba_for(tH, tL):
        yhatH = (sH >= tH).astype(int)
        yhatL = (sL >= tL).astype(int)
        y_true = np.concatenate([yH, yL])
        y_pred = np.concatenate([yhatH, yhatL])
        return _ba_binary(y_true, y_pred)

    candH = _grid(0.0, 1.0, COARSE_STEPS)
    candL = _grid(0.0, 1.0, COARSE_STEPS)
    best = {"BA_val": -np.inf, "tauH": None, "tauL": None}
    for tH in candH:
        ba_vec = np.empty_like(candL)
        for j, tL in enumerate(candL):
            ba_vec[j] = _pooled_ba_for(tH, tL)
        jmax = int(np.nanargmax(ba_vec))
        if float(ba_vec[jmax]) > best["BA_val"]:
            best.update({"BA_val": float(ba_vec[jmax]), "tauH": float(tH), "tauL": float(candL[jmax])})

    # fine
    lH = max(0.0, best["tauH"] - FINE_MARGIN); rH = min(1.0, best["tauH"] + FINE_MARGIN)
    lL = max(0.0, best["tauL"] - FINE_MARGIN); rL = min(1.0, best["tauL"] + FINE_MARGIN)
    candH2 = _grid(lH, rH, FINE_STEPS); candL2 = _grid(lL, rL, FINE_STEPS)

    best2 = dict(best)
    for tH in candH2:
        ba_vec2 = np.empty_like(candL2)
        for j, tL in enumerate(candL2):
            ba_vec2[j] = _pooled_ba_for(tH, tL)
        jmax = int(np.nanargmax(ba_vec2))
        if float(ba_vec2[jmax]) > best2["BA_val"]:
            best2.update({"BA_val": float(ba_vec2[jmax]), "tauH": float(tH), "tauL": float(candL2[jmax])})

    return best2  # {"tauH","tauL","BA_val"}

# ---------------- outer LOSO with inner concatenation ----------------
logo_outer = LeaveOneGroupOut()
rows, pred_rows = [], []

for fold_id, (tr_idx, te_idx) in enumerate(logo_outer.split(X_k, y_base.values, g_base.values), start=1):
    train_mask = pd.Series(False, index=g_base.index); train_mask.iloc[tr_idx] = True
    test_mask  = pd.Series(False, index=g_base.index);  test_mask.iloc[te_idx]  = True
    test_sid = g_base.iloc[te_idx].iloc[0]

    # ===== inner folds =====
    inner_ids   = sorted(g_base[train_mask].unique())
    inner_folds = choose_inner_folds_loso(inner_ids)
    if VERBOSE:
        print(f"[inner folds] {len(inner_folds)} splits -> val subjects = {', '.join(inner_ids)}")

    # 連結valの器
    val_scores_all, val_y_all, val_grp_all = [], [], []

    # --- inner train に両群が居るか（群別τの前提） ---
    inner_train_groups = fair_groups[train_mask]
    if not (("High" in set(inner_train_groups)) and ("Low" in set(inner_train_groups))):
        raise RuntimeError(f"[Cell5A] fold{fold_id}: inner-train に両群(High/Low)が必要（群別τの前提）")

    # ===== inner: train→val 予測の連結 =====
    for inner_val in inner_folds:
        val_mask  = g_base.isin(inner_val) & train_mask
        trn_mask  = train_mask & (~val_mask)
        if not trn_mask.any() or not val_mask.any():
            continue

        X_tr, y_tr = X_k[trn_mask], y_base[trn_mask]
        X_vl, y_vl = X_k[val_mask], y_base[val_mask]
        grp_vl     = fair_groups[val_mask].to_numpy()

        # inner 学習
        model_inner = fit_classifier(X_tr, y_tr)
        sc_vl = predict_positive_score(model_inner, X_vl).astype(float)

        # 連結（生の val 予測）
        val_scores_all.append(sc_vl)
        val_y_all.append(y_vl.to_numpy())
        val_grp_all.append(grp_vl)

        if VERBOSE:
            nH = int((grp_vl=="High").sum()); nL = int((grp_vl=="Low").sum())
            print(f"[DBG] fold{fold_id}: inner_val size={len(y_vl):3d}  High={nH} Low={nL}")

    # 連結
    if len(val_scores_all) == 0:
        raise RuntimeError(f"[Cell5A] fold{fold_id}: inner validation が空である．")
    s_val = np.concatenate(val_scores_all)
    y_val = np.concatenate(val_y_all)
    g_val = np.concatenate(val_grp_all)

    # ===== 連結valで一度だけ τ を最適化（BA） =====
    res_single = _single_tau_opt(s_val, y_val)
    tau_single = float(res_single["tau"])

    res_wg = _wg_ba_opt_joint(s_val, y_val, g_val)
    tauH_wg, tauL_wg = float(res_wg["tauH"]), float(res_wg["tauL"])

    res_group = _group_ba_opt(s_val, y_val, g_val)
    tauH_grp, tauL_grp = float(res_group["tauH"]), float(res_group["tauL"])

    if VERBOSE:
        print(f"[DBG] fold{fold_id}: tau_single={tau_single:.6f} | "
              f"Group-BA(tH,tL)=({tauH_grp:.6f},{tauL_grp:.6f}) | "
              f"WG-BA(tH,tL)=({tauH_wg:.6f},{tauL_wg:.6f})")

    # ===== outer test =====
    X_tr_o, y_tr_o = X_k[train_mask], y_base[train_mask]
    X_te_o, y_te_o = X_k[test_mask],  y_base[test_mask]
    grp_te         = fair_groups[test_mask].to_numpy()

    model_outer = fit_classifier(X_tr_o, y_tr_o)
    sc_te = predict_positive_score(model_outer, X_te_o).astype(float)

    yhat_single = (sc_te >= tau_single).astype(int)
    yhat_groupBA = (sc_te >= np.where(grp_te=="High", tauH_grp, tauL_grp)).astype(int)
    yhat_wgBA    = (sc_te >= np.where(grp_te=="High", tauH_wg,  tauL_wg )).astype(int)

    BA_single = _ba_binary(y_te_o.to_numpy(), yhat_single)
    BA_group  = _ba_binary(y_te_o.to_numpy(), yhat_groupBA)
    BA_wg     = _ba_binary(y_te_o.to_numpy(), yhat_wgBA)

    if VERBOSE:
        same_grp = bool(np.array_equal(yhat_single, yhat_groupBA))
        same_wg  = bool(np.array_equal(yhat_single, yhat_wgBA))
        nH_te = int((grp_te=="High").sum()); nL_te = int((grp_te=="Low").sum())
        print(f"[DBG] fold{fold_id}: TEST High={nH_te} Low={nL_te} | "
              f"Single==Group-BA: {same_grp}  Single==WG-BA: {same_wg} | "
              f"BA(single, group, wg)=({BA_single:.3f}, {BA_group:.3f}, {BA_wg:.3f})")

    rows.append({
        "fold_id": int(fold_id),
        "test_id": str(test_sid),
        "best_k": int(best_k),
        "tau_single": float(tau_single),
        "tau_high_GroupBA": float(tauH_grp), "tau_low_GroupBA": float(tauL_grp),
        "tau_high_WGBA": float(tauH_wg),     "tau_low_WGBA": float(tauL_wg),
        "BA_single": float(BA_single),
        "BA_group":  float(BA_group),
        "BA_group_WG": float(BA_wg),
        "n_test": int(len(y_te_o)),
    })

    # 予測詳細（BA専用命名）
    for yy, ss, gg, ys, yg, yw in zip(y_te_o, sc_te, grp_te, yhat_single, yhat_groupBA, yhat_wgBA):
        pred_rows.append({
            "fold_id": int(fold_id),
            "test_id": str(test_sid),
            "y_true": int(yy),
            "proba": float(ss),
            "group": str(gg),
            "y_pred_single": int(ys),
            "y_pred_group_BA": int(yg),
            "y_pred_group_WG": int(yw),
        })
    
    print(f"[DBG] fold{fold_id}: WG-joint val -> "
          f"BA_H={res_wg['BA_H_val']:.3f}, BA_L={res_wg['BA_L_val']:.3f}, "
          f"WG(BA)={res_wg['WG_BA_val']:.3f}, pooled(BA)={res_wg['BA_pooled_val']:.3f}, "
          f"(tH,tL)=({tauH_wg:.4f},{tauL_wg:.4f})")

# ---------------- 出力 ----------------
df_fold = pd.DataFrame(rows)
df_pred = pd.DataFrame(pred_rows)
df_fold.to_csv(outpath("GROUP_AWARE_THRESH_BY_FOLD.CSV"), index=False, encoding="utf-8-sig")
df_pred.to_csv(outpath("GROUP_AWARE_PREDICTIONS.CSV"), index=False, encoding="utf-8-sig")

# 参考: プールAUC（確率は共通なので1つ）
auc_pool = float(roc_auc_score(df_pred["y_true"].to_numpy(), df_pred["proba"].to_numpy()))

def _pooled_ba(col):
    y_true = df_pred["y_true"].to_numpy()
    yhat   = df_pred[col].to_numpy().astype(int)
    return _ba_binary(y_true, yhat)

summary = {
    "best_k": int(best_k),
    "AUC_pooled": auc_pool,  # 不要ならこのキーごと削除可
    "BA_pooled_single": _pooled_ba("y_pred_single"),
    "BA_pooled_group":  _pooled_ba("y_pred_group_BA"),
    "BA_pooled_group_WG": _pooled_ba("y_pred_group_WG"),
    "metric": "BA",
    "n_samples": int(len(df_pred)),
    "n_pos": int((df_pred["y_true"]==1).sum()),
    "n_neg": int((df_pred["y_true"]==0).sum()),
}
pd.DataFrame([summary]).to_csv(outpath("GROUP_AWARE_SUMMARY.CSV"), index=False, encoding="utf-8-sig")
print(f"[Cell5A] Done: outer folds={len(df_fold)}, pooled AUC={auc_pool:.3f}")
print(f"[Cell5A] Summary: {summary}")

# ---------------- 混同行列プロット（BA専用タイトル） ----------------
def _draw_cm(cm, title, path):
    fig, ax = plt.subplots(figsize=(7,6))
    im = ax.imshow(cm, cmap="Blues", vmin=0, vmax=max(cm.max(),1))
    labels = np.array([["TN","FP"],["FN","TP"]])
    for i in range(2):
        for j in range(2):
            val = int(cm[i,j])
            color = "white" if val > 0.6*im.get_clim()[1] else "black"
            ax.text(j, i, f"{labels[i,j]}\n{val}", ha="center", va="center",
                    fontsize=18, fontweight="bold", color=color)
    ax.set_xticks([0,1]); ax.set_xticklabels(["Pred:0","Pred:1"])
    ax.set_yticks([0,1]); ax.set_yticklabels(["True:0","True:1"], rotation=90, va="center")
    ax.set_title(title); ax.grid(False)
    plt.tight_layout(); plt.savefig(path, dpi=300); plt.close()

y_pool = df_pred["y_true"].to_numpy()
cm_single = skm.confusion_matrix(y_pool, df_pred["y_pred_single"],   labels=[0,1])
cm_group  = skm.confusion_matrix(y_pool, df_pred["y_pred_group_BA"], labels=[0,1])
cm_wg     = skm.confusion_matrix(y_pool, df_pred["y_pred_group_WG"], labels=[0,1])

_draw_cm(cm_single, f"Cell5A Single τ (BA={summary['BA_pooled_single']:.3f})",
         outpath("CONFMAT_CELL5A_SINGLE(BA).png"))
_draw_cm(cm_group,  f"Cell5A Group-BA τ (BA={summary['BA_pooled_group']:.3f})",
         outpath("CONFMAT_CELL5A_GROUP(BA).png"))
_draw_cm(cm_wg,     f"Cell5A WG-BA τ (BA={summary['BA_pooled_group_WG']:.3f})",
         outpath("CONFMAT_CELL5A_WG(BA).png"))
print("[Cell5A] Confusion matrices saved -> CONFMAT_CELL5A_*.png")


In [ ]:
# ===== Cell 5B++: 確率スコア分布（OVERALL/群別/各Fold）＋ τ の中央値/IQR 帯・±表記 =====
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ---------- 設定 ----------
BINS = 40
LW = 1.5
FS_TITLE, FS_LABEL, FS_LEGEND, FS_TICK = 30, 24, 20, 20

# 色指定（ユーザ指定）
COLOR_SICK = "red"   # True:Sick
COLOR_NON  = "blue"  # True:Non-Sick

# 線色（しきい値）
COLOR_SINGLE = "black"
COLOR_GROUP  = "green"
COLOR_WG     = "purple"

# 出力フォルダ（新規作成）
RUN_ROOT = os.path.dirname(outpath("__dummy__"))
IMG_DIR  = os.path.join(RUN_ROOT, "PROBA_DIST_BA")
os.makedirs(IMG_DIR, exist_ok=True)

# ファイルパス
SAVE_OVERALL_SvG   = os.path.join(IMG_DIR, "OVERALL_SvGroup.png")
SAVE_OVERALL_SvWG  = os.path.join(IMG_DIR, "OVERALL_SvWG.png")
SAVE_BYGROUP_SvG   = os.path.join(IMG_DIR, "BYGROUP_SvGroup.png")
SAVE_BYGROUP_SvWG  = os.path.join(IMG_DIR, "BYGROUP_SvWG.png")

# ---------- 入力 ----------
pred_path = outpath("GROUP_AWARE_PREDICTIONS.CSV")
fold_path = outpath("GROUP_AWARE_THRESH_BY_FOLD.CSV")
if not (os.path.exists(pred_path) and os.path.exists(fold_path)):
    raise FileNotFoundError("[Cell5B++] 必要CSVが見つからない（Cell 5A を先に実行）")

df_pred = pd.read_csv(pred_path, encoding="utf-8-sig")
df_fold = pd.read_csv(fold_path, encoding="utf-8-sig")

# ---------- モード自動判定（F1 or BA） ----------
cols_f1 = {"high":"tau_high_GroupF1", "low":"tau_low_GroupF1", "wgh":"tau_high_WGF1", "wgl":"tau_low_WGF1"}
cols_ba = {"high":"tau_high_GroupBA", "low":"tau_low_GroupBA", "wgh":"tau_high_WGBA", "wgl":"tau_low_WGBA"}

if all(c in df_fold.columns for c in [cols_f1["high"], cols_f1["low"]]):
    mode = "F1"
    c_high, c_low, c_wgh, c_wgl = cols_f1["high"], cols_f1["low"], cols_f1["wgh"], cols_f1["wgl"]
    pred_group_col = "y_pred_group_F1"
elif all(c in df_fold.columns for c in [cols_ba["high"], cols_ba["low"]]):
    mode = "BA"
    c_high, c_low, c_wgh, c_wgl = cols_ba["high"], cols_ba["low"], cols_ba["wgh"], cols_ba["wgl"]
    pred_group_col = "y_pred_group_BA"
else:
    raise RuntimeError("[Cell5B++] しきい値列が見つからない（F1/BAどちらかのCell 5Aの出力が必要）")

# ---------- 集約: 中央値/IQR（Q1〜Q3） ----------
def _qstats(s):
    s = pd.to_numeric(s, errors="coerce")
    s = s[np.isfinite(s)]
    if s.size == 0:
        return np.nan, np.nan, np.nan, np.nan  # med, q1, q3, half_iqr
    med = float(np.nanmedian(s))
    q1, q3 = np.nanpercentile(s, [25, 75])
    half = float((q3 - q1)/2.0)
    return float(med), float(q1), float(q3), half

tau_single_med, tau_single_q1, tau_single_q3, tau_single_half = _qstats(df_fold["tau_single"])
tau_high_med,   tau_high_q1,   tau_high_q3,   tau_high_half   = _qstats(df_fold[c_high])
tau_low_med,    tau_low_q1,    tau_low_q3,    tau_low_half    = _qstats(df_fold[c_low])
tau_high_wg,    tau_high_wg_q1, tau_high_wg_q3, tau_high_wg_half = _qstats(df_fold[c_wgh]) if c_wgh in df_fold.columns else (np.nan, np.nan, np.nan, np.nan)
tau_low_wg,     tau_low_wg_q1,  tau_low_wg_q3,  tau_low_wg_half  = _qstats(df_fold[c_wgl]) if c_wgl in df_fold.columns else (np.nan, np.nan, np.nan, np.nan)

# ---------- データ分解 ----------
proba = pd.to_numeric(df_pred["proba"], errors="coerce").values
ytrue = pd.to_numeric(df_pred["y_true"], errors="coerce").values.astype(int)
grp   = df_pred["group"].astype(str).str.strip()

p_sick = proba[ytrue == 1]   # True:Sick
p_non  = proba[ytrue == 0]   # True:Non-Sick
n_sick, n_non = len(p_sick), len(p_non)

maskH = (grp == "High"); maskL = (grp == "Low")

# ---------- ユーティリティ ----------
def _style_axes(ax, title=None):
    if title: ax.set_title(title, fontsize=FS_TITLE)
    ax.set_xlabel("Predicted probability", fontsize=FS_LABEL)
    ax.set_ylabel("Density", fontsize=FS_LABEL)
    ax.tick_params(axis="both", labelsize=FS_TICK)
    ax.set_xlim(0, 1)

def _hist_overall(ax):
    ax.hist(p_sick, bins=BINS, density=True, alpha=0.5, label=f"True:Sick (n={n_sick})", color=COLOR_SICK)
    ax.hist(p_non,  bins=BINS, density=True, alpha=0.5, label=f"True:Non-Sick (n={n_non})", color=COLOR_NON)

def _hist_bygroup(axes):
    # High
    p_sick_H = proba[(ytrue==1) & maskH]; p_non_H = proba[(ytrue==0) & maskH]
    axes[0].hist(p_sick_H, bins=BINS, density=True, alpha=0.5, label=f"True:Sick (n={len(p_sick_H)})", color=COLOR_SICK)
    axes[0].hist(p_non_H,  bins=BINS, density=True, alpha=0.5, label=f"True:Non-Sick (n={len(p_non_H)})", color=COLOR_NON)
    _style_axes(axes[0], "High group")
    # Low
    p_sick_L = proba[(ytrue==1) & maskL]; p_non_L = proba[(ytrue==0) & maskL]
    axes[1].hist(p_sick_L, bins=BINS, density=True, alpha=0.5, label=f"True:Sick (n={len(p_sick_L)})", color=COLOR_SICK)
    axes[1].hist(p_non_L,  bins=BINS, density=True, alpha=0.5, label=f"True:Non-Sick (n={len(p_non_L)})", color=COLOR_NON)
    _style_axes(axes[1], "Low group")

def _vline_with_iqr(ax, x_med, q1, q3, color, ls, label_core):
    if np.isfinite(x_med):
        ax.axvline(x_med, color=color, linestyle=ls, linewidth=LW,
                   label=f"{label_core} = {x_med:.3f} ± {(q3-q1)/2:.3f}" if (np.isfinite(q1) and np.isfinite(q3)) else f"{label_core} = {x_med:.3f}")
    if np.isfinite(q1) and np.isfinite(q3):
        ax.axvspan(q1, q3, color=color, alpha=0.12)

# ---------- 1) OVERALL: Single vs Group（中央値・IQR帯・±表記） ----------
fig, ax = plt.subplots(figsize=(9,6))
_hist_overall(ax)
_vline_with_iqr(ax, tau_single_med, tau_single_q1, tau_single_q3, COLOR_SINGLE, "-", "τ_single")
_vline_with_iqr(ax, tau_high_med,   tau_high_q1,   tau_high_q3,   COLOR_GROUP,  "--", f"τ_high_{mode}")
_vline_with_iqr(ax, tau_low_med,    tau_low_q1,    tau_low_q3,    COLOR_GROUP,  "--", f"τ_low_{mode}")
_style_axes(ax, title=f"Probability distribution (OVERALL) — Single vs Group [{mode}]")
ax.legend(fontsize=FS_LEGEND)
plt.tight_layout(); plt.savefig(SAVE_OVERALL_SvG, dpi=300); plt.close()
print(f"[Cell5B++] Saved -> {SAVE_OVERALL_SvG}")

# ---------- 2) OVERALL: Single vs WG（中央値・IQR帯・±表記） ----------
fig, ax = plt.subplots(figsize=(9,6))
_hist_overall(ax)
_vline_with_iqr(ax, tau_single_med, tau_single_q1, tau_single_q3, COLOR_SINGLE, "-", "τ_single")
_vline_with_iqr(ax, tau_high_wg,    tau_high_wg_q1, tau_high_wg_q3, COLOR_WG, ":", "τ_high_WG")
_vline_with_iqr(ax, tau_low_wg,     tau_low_wg_q1,  tau_low_wg_q3,  COLOR_WG, ":", "τ_low_WG")
_style_axes(ax, title=f"Probability distribution (OVERALL) — Single vs WG [{mode}]")
ax.legend(fontsize=FS_LEGEND)
plt.tight_layout(); plt.savefig(SAVE_OVERALL_SvWG, dpi=300); plt.close()
print(f"[Cell5B++] Saved -> {SAVE_OVERALL_SvWG}")

# ---------- 3) BY_GROUP: Single vs Group（各群パネルに該当の IQR帯） ----------
fig, axes = plt.subplots(2, 1, figsize=(9,10), sharex=True)
_hist_bygroup(axes)
# Single は両段に表示
_vline_with_iqr(axes[0], tau_single_med, tau_single_q1, tau_single_q3, COLOR_SINGLE, "-", "τ_single")
_vline_with_iqr(axes[1], tau_single_med, tau_single_q1, tau_single_q3, COLOR_SINGLE, "-", "τ_single")
# High 段に High の Group τ、Low 段に Low の Group τ
_vline_with_iqr(axes[0], tau_high_med, tau_high_q1, tau_high_q3, COLOR_GROUP, "--", f"τ_high_{mode}")
_vline_with_iqr(axes[1], tau_low_med,  tau_low_q1,  tau_low_q3,  COLOR_GROUP, "--", f"τ_low_{mode}")
axes[0].legend(fontsize=FS_LEGEND); axes[1].legend(fontsize=FS_LEGEND)
plt.tight_layout(); plt.savefig(SAVE_BYGROUP_SvG, dpi=300); plt.close()
print(f"[Cell5B++] Saved -> {SAVE_BYGROUP_SvG}")

# ---------- 4) BY_GROUP: Single vs WG（各群パネルに該当の IQR帯） ----------
fig, axes = plt.subplots(2, 1, figsize=(9,10), sharex=True)
_hist_bygroup(axes)
_vline_with_iqr(axes[0], tau_single_med, tau_single_q1, tau_single_q3, COLOR_SINGLE, "-", "τ_single")
_vline_with_iqr(axes[1], tau_single_med, tau_single_q1, tau_single_q3, COLOR_SINGLE, "-", "τ_single")
_vline_with_iqr(axes[0], tau_high_wg, tau_high_wg_q1, tau_high_wg_q3, COLOR_WG, ":", "τ_high_WG")
_vline_with_iqr(axes[1], tau_low_wg,  tau_low_wg_q1,  tau_low_wg_q3,  COLOR_WG, ":", "τ_low_WG")
axes[0].legend(fontsize=FS_LEGEND); axes[1].legend(fontsize=FS_LEGEND)
plt.tight_layout(); plt.savefig(SAVE_BYGROUP_SvWG, dpi=300); plt.close()
print(f"[Cell5B++] Saved -> {SAVE_BYGROUP_SvWG}")

# ---------- 5) Fold単位：OVERALL の確率分布と各Foldの τ（Single vs Group / Single vs WG） ----------
#   - 各Foldのテストサンプルのみでヒストを作図
#   - しきい値は「そのFold行」の値を直接表示（±は不要／IQR帯は使わない）
for _, row in df_fold.iterrows():
    fid = int(row["fold_id"]) if "fold_id" in row else None
    test_id = str(row.get("test_id", f"fold{fid}"))
    sub = df_pred[df_pred["fold_id"] == fid] if "fold_id" in df_pred.columns and fid is not None else df_pred.copy()

    p = pd.to_numeric(sub["proba"], errors="coerce").values
    yt = pd.to_numeric(sub["y_true"], errors="coerce").values.astype(int)
    p_s, p_n = p[yt==1], p[yt==0]

    # 値（このFoldの τ）
    t_single = float(row["tau_single"])
    t_high   = float(row[c_high]) if c_high in row else np.nan
    t_low    = float(row[c_low])  if c_low  in row else np.nan
    t_high_w = float(row[c_wgh])  if c_wgh  in row else np.nan
    t_low_w  = float(row[c_wgl])  if c_wgl  in row else np.nan

    # 出力パス
    p_sg  = os.path.join(IMG_DIR, f"FOLD{fid:02d}_{test_id}_OVERALL_SvGroup.png")
    p_wg  = os.path.join(IMG_DIR, f"FOLD{fid:02d}_{test_id}_OVERALL_SvWG.png")

    # Single vs Group
    fig, ax = plt.subplots(figsize=(9,6))
    ax.hist(p_s, bins=BINS, density=True, alpha=0.5, label=f"True:Sick (n={len(p_s)})", color=COLOR_SICK)
    ax.hist(p_n, bins=BINS, density=True, alpha=0.5, label=f"True:Non-Sick (n={len(p_n)})", color=COLOR_NON)
    if np.isfinite(t_single): ax.axvline(t_single, color=COLOR_SINGLE, linestyle="-", linewidth=LW, label=f"τ_single = {t_single:.3f}")
    if np.isfinite(t_high):   ax.axvline(t_high,   color=COLOR_GROUP,  linestyle="--", linewidth=LW, label=f"τ_high_{mode} = {t_high:.3f}")
    if np.isfinite(t_low):    ax.axvline(t_low,    color=COLOR_GROUP,  linestyle="--", linewidth=LW, label=f"τ_low_{mode}  = {t_low:.3f}")
    _style_axes(ax, title=f"[Fold {fid}] OVERALL — Single vs Group [{mode}]  (test={test_id})")
    ax.legend(fontsize=FS_LEGEND)
    plt.tight_layout(); plt.savefig(p_sg, dpi=300); plt.close()
    print(f"[Cell5B++] Saved -> {p_sg}")

    # Single vs WG
    fig, ax = plt.subplots(figsize=(9,6))
    ax.hist(p_s, bins=BINS, density=True, alpha=0.5, label=f"True:Sick (n={len(p_s)})", color=COLOR_SICK)
    ax.hist(p_n, bins=BINS, density=True, alpha=0.5, label=f"True:Non-Sick (n={len(p_n)})", color=COLOR_NON)
    if np.isfinite(t_single): ax.axvline(t_single, color=COLOR_SINGLE, linestyle="-", linewidth=LW, label=f"τ_single = {t_single:.3f}")
    if np.isfinite(t_high_w): ax.axvline(t_high_w, color=COLOR_WG,     linestyle=":", linewidth=LW, label=f"τ_high_WG = {t_high_w:.3f}")
    if np.isfinite(t_low_w):  ax.axvline(t_low_w,  color=COLOR_WG,     linestyle=":", linewidth=LW, label=f"τ_low_WG  = {t_low_w:.3f}")
    _style_axes(ax, title=f"[Fold {fid}] OVERALL — Single vs WG [{mode}]  (test={test_id})")
    ax.legend(fontsize=FS_LEGEND)
    plt.tight_layout(); plt.savefig(p_wg, dpi=300); plt.close()
    print(f"[Cell5B++] Saved -> {p_wg}")

print(f"[Cell5B++] All images saved in: {IMG_DIR}")
